# 🐍 Python & NumPy Fundamentals for Neuroscience

Welcome back, neural network explorer! 🧠⚡

Before we dive into the exciting world of brain signal analysis, let's make sure you're comfortable with the core tools you'll be using every day. This notebook is designed as a **refresher** - not a complete introduction to programming.

## What We'll Cover Today 🎯

1. **Python Essentials**: The language constructs you'll use constantly
2. **NumPy Fundamentals**: The backbone of scientific computing
3. **Data Structures**: Arrays, lists, and when to use each
4. **Mathematical Operations**: Linear algebra for neural networks
5. **Brain Signal Examples**: Real-world applications of what you're learning

## 💡 Why This Matters

In neuroscience, you'll constantly work with:
- **Multi-dimensional arrays** (time × channels × trials)
- **Mathematical operations** (filtering, Fourier transforms, matrix multiplication)
- **Data manipulation** (selecting time windows, averaging across trials)
- **Efficient computation** (vectorized operations for speed)

Master these fundamentals, and you'll breeze through the advanced topics!

---

*Ready to refresh your Python skills? Let's go! 🚀*

# 🛠️ Setting Up Our Environment

First, let's import the libraries we'll be using and set up some nice defaults for visualization.

In [ ]:
# Essential imports for today's session
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal
import time
import warnings
import sys
import psutil
import platform
from IPython.display import display, HTML, clear_output

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Enhanced matplotlib setup
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

# NumPy settings for cleaner output
np.set_printoptions(precision=3, suppress=True)

# Check system capabilities
def check_system_capabilities():
    """Check system resources and provide recommendations"""
    
    print("🔍 System Capabilities Check")
    print("=" * 40)
    
    # Memory check
    try:
        memory_info = psutil.virtual_memory()
        total_memory_gb = memory_info.total / (1024**3)
        available_memory_gb = memory_info.available / (1024**3)
        
        print(f"💾 Total RAM: {total_memory_gb:.1f} GB")
        print(f"💾 Available RAM: {available_memory_gb:.1f} GB")
        
        if total_memory_gb < 4:
            print("⚠️ Low RAM detected. Consider using smaller datasets.")
        elif total_memory_gb >= 8:
            print("✅ Good RAM capacity for large neuroscience datasets.")
        
    except Exception as e:
        print(f"ℹ️ Memory info unavailable: {e}")
    
    # CPU check
    try:
        cpu_count = psutil.cpu_count()
        print(f"🔥 CPU Cores: {cpu_count}")
        
        if cpu_count >= 4:
            print("✅ Multi-core processing available for parallel operations.")
        else:
            print("ℹ️ Single/dual-core system. Some operations may be slower.")
            
    except Exception as e:
        print(f"ℹ️ CPU info unavailable: {e}")
    
    # Platform-specific recommendations
    system = platform.system()
    print(f"💻 Platform: {system}")
    
    if system == "Windows":
        print("💡 Windows tip: Use WSL2 for better Linux compatibility")
    elif system == "Darwin":
        print("💡 macOS tip: Consider using conda for package management")
    elif system == "Linux":
        print("✅ Linux detected - optimal for scientific computing")
    
    print("=" * 40)
    return True

# Error handling decorator
def handle_errors(func):
    """Decorator to handle common errors gracefully"""
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except ImportError as e:
            print(f"❌ Import Error: {e}")
            print("💡 Solution: Install missing packages or restart kernel")
            return None
        except MemoryError:
            print("❌ Memory Error: Dataset too large for available RAM")
            print("💡 Solution: Use smaller datasets or increase system memory")
            return None
        except Exception as e:
            print(f"❌ Unexpected Error: {e}")
            print("💡 Solution: Check input parameters and data types")
            return None
    return wrapper

# Enhanced imports with error handling
@handle_errors
def import_interactive_widgets():
    """Import interactive widgets with graceful fallback"""
    try:
        import ipywidgets as widgets
        from ipywidgets import interact, interactive, fixed
        print("✅ Interactive widgets available")
        return widgets, interact, interactive, fixed
    except ImportError:
        print("ℹ️ Interactive widgets not available (install ipywidgets)")
        return None, None, None, None

# Run system check
check_system_capabilities()

# Try to import interactive widgets
widgets, interact, interactive, fixed = import_interactive_widgets()

print("\n✅ Environment ready!")
print(f"📊 NumPy version: {np.__version__}")
print(f"📈 Matplotlib version: {plt.matplotlib.__version__}")
print(f"🔬 SciPy version: {scipy.__version__}")
print(f"🐍 Python version: {sys.version.split()[0]}")

# Create a simple neuroscience glossary
neuroscience_glossary = {
    'EEG': 'Electroencephalography - recording electrical activity of the brain',
    'Alpha waves': 'Brain waves (8-13 Hz) associated with relaxed awareness',
    'Beta waves': 'Brain waves (13-30 Hz) associated with active concentration',
    'Theta waves': 'Brain waves (4-8 Hz) associated with creativity and deep meditation',
    'Delta waves': 'Brain waves (0.5-4 Hz) associated with deep sleep',
    'Gamma waves': 'Brain waves (30-100 Hz) associated with high-level cognitive processing',
    'Artifact': 'Unwanted signal contamination in neural recordings',
    'Epoch': 'A segment of continuous EEG data, typically around a stimulus event',
    'Montage': 'The arrangement of electrodes on the scalp',
    'Coherence': 'A measure of synchronization between two brain signals'
}

print("\n📚 Neuroscience Glossary loaded (access via 'neuroscience_glossary')")
print("💡 Example: print(neuroscience_glossary['EEG'])")

# Performance benchmark function
def benchmark_numpy_performance():
    """Benchmark NumPy performance on this system"""
    
    print("\n🏃 NumPy Performance Benchmark")
    print("=" * 35)
    
    # Test array creation
    start_time = time.time()
    large_array = np.random.randn(1000, 1000)
    creation_time = time.time() - start_time
    print(f"Array creation (1M elements): {creation_time:.4f} seconds")
    
    # Test matrix multiplication
    start_time = time.time()
    result = np.dot(large_array, large_array.T)
    multiplication_time = time.time() - start_time
    print(f"Matrix multiplication: {multiplication_time:.4f} seconds")
    
    # Test FFT (important for signal processing)
    signal_1d = np.random.randn(10000)
    start_time = time.time()
    fft_result = np.fft.fft(signal_1d)
    fft_time = time.time() - start_time
    print(f"FFT (10K samples): {fft_time:.4f} seconds")
    
    # Performance assessment
    if multiplication_time < 0.1:
        print("✅ Excellent NumPy performance!")
    elif multiplication_time < 0.5:
        print("✅ Good NumPy performance")
    else:
        print("⚠️ Slower NumPy performance - consider updating NumPy/BLAS")
    
    print("=" * 35)
    return creation_time, multiplication_time, fft_time

# Run performance benchmark
benchmark_results = benchmark_numpy_performance()

print("\n🎯 Ready to explore Python & NumPy for Neuroscience!")
print("💡 Tip: Use 'help(function_name)' to get detailed documentation")

# 🐍 Python Essentials: Quick Review

Let's quickly review the Python constructs you'll use most often in neuroscience computing.

## List Comprehensions: Your Secret Weapon 🔧

List comprehensions are incredibly useful for data processing. In neuroscience, you'll often need to apply operations across multiple files, channels, or trials.

In [ ]:
# 🔧 Common Errors & Debugging Like a Neuroscientist

print("🚨 Common Python/NumPy Errors in Neuroscience & How to Fix Them")
print("=" * 60)

# Create examples of common errors with solutions
def demonstrate_common_errors():
    """Show common errors and their solutions with explanations"""
    
    print("\n1. 📊 INDEXING ERRORS")
    print("-" * 20)
    
    # Example data (channels × time × trials)
    eeg_data = np.random.randn(64, 1000, 100)
    channels = [0, 1, 2]
    trials = [10, 20, 30, 40, 50]
    
    print("❌ WRONG - Broadcasting Error:")
    print("   eeg_data[channels, :, trials]  # This will fail!")
    
    try:
        wrong_result = eeg_data[channels, :, trials]
    except IndexError as e:
        print(f"   Error: {e}")
        print("\n💡 SOLUTION:")
        print("   eeg_data[np.ix_(channels, range(eeg_data.shape[1]), trials)]")
        
        # Correct approach
        correct_result = eeg_data[np.ix_(channels, range(eeg_data.shape[1]), trials)]
        print(f"   ✅ Result shape: {correct_result.shape}")
    
    print("\n2. 🧮 MATRIX DIMENSION ERRORS")
    print("-" * 30)
    
    print("❌ WRONG - Shape Mismatch:")
    print("   np.dot(matrix_64x1000, matrix_100x64)  # Incompatible shapes!")
    
    matrix_a = np.random.randn(64, 1000)
    matrix_b = np.random.randn(100, 64)
    
    try:
        wrong_multiplication = np.dot(matrix_a, matrix_b)
    except ValueError as e:
        print(f"   Error: {e}")
        print("\n💡 SOLUTION:")
        print("   Check dimensions: (64,1000) @ (1000,X) ✅")
        print("   Use matrix_a @ matrix_b.T or matrix_a.T @ matrix_b")
        
        # Correct approaches
        correct_mult1 = matrix_a @ matrix_b.T  # (64,1000) @ (1000,100) = (64,100)
        correct_mult2 = matrix_a.T @ matrix_b.T  # (1000,64) @ (64,100) = (1000,100)
        print(f"   ✅ Result shapes: {correct_mult1.shape}, {correct_mult2.shape}")
    
    print("\n3. 🔢 DATA TYPE ERRORS")
    print("-" * 20)
    
    print("❌ WRONG - Integer Division (Python 2 style):")
    sampling_rate = 250
    duration = 3
    
    # This can cause issues with indexing
    wrong_samples = duration * sampling_rate / 2  # Float result
    print(f"   wrong_samples = {wrong_samples} (type: {type(wrong_samples)})")
    
    print("\n💡 SOLUTION:")
    correct_samples = int(duration * sampling_rate // 2)  # Integer result
    print(f"   correct_samples = {correct_samples} (type: {type(correct_samples)})")
    
    print("\n4. 📏 AXIS CONFUSION")
    print("-" * 15)
    
    # Example: EEG data (channels × time × trials)
    eeg_example = np.random.randn(8, 1000, 50)
    
    print("❌ WRONG - Unclear axis operations:")
    wrong_mean = np.mean(eeg_example, axis=1)  # What does this average?
    print(f"   np.mean(eeg_data, axis=1) → shape: {wrong_mean.shape}")
    print("   ❓ Is this averaging across time, channels, or trials?")
    
    print("\n💡 SOLUTION - Be explicit:")
    trial_average = np.mean(eeg_example, axis=2)  # Average across trials
    time_average = np.mean(eeg_example, axis=1)   # Average across time  
    channel_average = np.mean(eeg_example, axis=0) # Average across channels
    
    print(f"   Trial average: {trial_average.shape} (channels × time)")
    print(f"   Time average: {time_average.shape} (channels × trials)")  
    print(f"   Channel average: {channel_average.shape} (time × trials)")

# Demonstrate common errors
demonstrate_common_errors()

print("\n\n🛠️ DEBUGGING TOOLKIT")
print("=" * 25)

def debug_array_info(array, name="array"):
    """Comprehensive array debugging information"""
    print(f"\n🔍 Debug info for '{name}':")
    print(f"   Shape: {array.shape}")
    print(f"   Data type: {array.dtype}")
    print(f"   Memory usage: {array.nbytes / 1024:.1f} KB")
    print(f"   Min/Max: {array.min():.3f} / {array.max():.3f}")
    print(f"   Mean/Std: {array.mean():.3f} / {array.std():.3f}")
    print(f"   Contains NaN: {np.isnan(array).any()}")
    print(f"   Contains Inf: {np.isinf(array).any()}")
    
    # Check for common issues
    if np.isnan(array).any():
        nan_count = np.isnan(array).sum()
        print(f"   ⚠️ Warning: {nan_count} NaN values detected!")
    
    if np.isinf(array).any():
        inf_count = np.isinf(array).sum()
        print(f"   ⚠️ Warning: {inf_count} infinite values detected!")
    
    if array.std() == 0:
        print(f"   ⚠️ Warning: Zero variance - all values are identical!")

# Example usage
sample_eeg = np.random.randn(32, 500, 20)
debug_array_info(sample_eeg, "sample_EEG_data")

print("\n\n🎯 NEUROSCIENCE-SPECIFIC DEBUGGING")
print("=" * 40)

def validate_eeg_data(eeg_data, sampling_rate=250, expected_duration=None):
    """Validate EEG data for common neuroscience issues"""
    
    print("🧠 EEG Data Validation Report")
    print("-" * 30)
    
    # Basic checks
    if eeg_data.ndim != 3:
        print("❌ Expected 3D array (channels × time × trials)")
        return False
    
    n_channels, n_timepoints, n_trials = eeg_data.shape
    duration = n_timepoints / sampling_rate
    
    print(f"✅ Data shape: {n_channels} channels × {n_timepoints} timepoints × {n_trials} trials")
    print(f"✅ Duration: {duration:.2f} seconds at {sampling_rate} Hz")
    
    # Check for realistic values
    amplitude_range = eeg_data.max() - eeg_data.min()
    if amplitude_range > 1000:
        print(f"⚠️ Large amplitude range: {amplitude_range:.1f} μV (check for artifacts)")
    elif amplitude_range < 1:
        print(f"⚠️ Small amplitude range: {amplitude_range:.3f} μV (check scaling)")
    else:
        print(f"✅ Realistic amplitude range: {amplitude_range:.1f} μV")
    
    # Check for flat channels
    channel_stds = np.std(eeg_data, axis=(1, 2))
    flat_channels = np.where(channel_stds < 0.01)[0]
    if len(flat_channels) > 0:
        print(f"⚠️ Potentially flat channels: {flat_channels}")
    else:
        print("✅ All channels show activity")
    
    # Check for extreme outliers
    z_scores = np.abs((eeg_data - np.mean(eeg_data)) / np.std(eeg_data))
    extreme_outliers = np.sum(z_scores > 5)
    if extreme_outliers > n_timepoints * n_channels * n_trials * 0.001:  # More than 0.1%
        print(f"⚠️ Many extreme outliers detected: {extreme_outliers}")
    else:
        print("✅ Outlier levels normal")
    
    # Expected duration check
    if expected_duration and abs(duration - expected_duration) > 0.1:
        print(f"⚠️ Duration mismatch: expected {expected_duration}s, got {duration:.2f}s")
    
    return True

# Test the validator
sample_eeg_data = np.random.randn(64, 1250, 30) * 50  # 64 channels, 5 seconds at 250 Hz, 30 trials
validate_eeg_data(sample_eeg_data, sampling_rate=250, expected_duration=5.0)

print("\n\n💡 PREVENTION TIPS")
print("=" * 20)

prevention_tips = [
    "Always check array shapes before operations: print(array.shape)",
    "Use descriptive variable names: 'eeg_channels_time_trials' not 'data'",
    "Add assertions for critical assumptions: assert data.ndim == 3",
    "Use explicit axis parameters: np.mean(data, axis=2) not np.mean(data, 2)",
    "Validate data ranges: check for realistic μV values in EEG",
    "Test with small datasets first before scaling up",
    "Use debug_array_info() function when things go wrong",
    "Keep a backup of original data before preprocessing"
]

for i, tip in enumerate(prevention_tips, 1):
    print(f"{i:2d}. {tip}")

print("\n\n📞 WHEN TO ASK FOR HELP")
print("=" * 30)

help_scenarios = [
    "🔴 Memory errors with large datasets → Consider data chunking",
    "🔴 Unexpected NaN/Inf values → Check data preprocessing pipeline", 
    "🔴 Shape mismatches in matrix operations → Review linear algebra",
    "🔴 Performance issues → Profile code and optimize bottlenecks",
    "🔴 Results don't match literature → Validate methodology and units"
]

for scenario in help_scenarios:
    print(f"   {scenario}")

print("\n💭 Remember: Every expert was once a beginner who never gave up debugging! 🚀")

## Functions: Building Reusable Analysis Tools 🔨

In [ ]:
def calculate_power_spectrum(signal_data, sampling_rate=250, freq_bands=None):
    """
    Calculate power spectrum for EEG-like data.
    
    Parameters:
    -----------
    signal_data : array-like
        Time series data
    sampling_rate : int
        Sampling frequency in Hz
    freq_bands : dict, optional
        Dictionary of frequency bands to analyze
        
    Returns:
    --------
    frequencies : array
        Frequency values
    power : array
        Power spectral density
    """
    if freq_bands is None:
        freq_bands = {
            'delta': (0.5, 4),
            'theta': (4, 8),
            'alpha': (8, 13),
            'beta': (13, 30)
        }
    
    # Calculate power spectrum using Welch's method
    frequencies, power = signal.welch(signal_data, sampling_rate, nperseg=sampling_rate*2)
    
    # Calculate band powers
    band_powers = {}
    for band, (low, high) in freq_bands.items():
        band_mask = (frequencies >= low) & (frequencies <= high)
        band_powers[band] = np.trapz(power[band_mask], frequencies[band_mask])
    
    return frequencies, power, band_powers

# Test our function
test_signal = np.random.randn(1000) + 2*np.sin(2*np.pi*10*np.linspace(0, 4, 1000))
freqs, psd, bands = calculate_power_spectrum(test_signal)

print("Band powers:")
for band, power in bands.items():
    print(f"  {band}: {power:.3f}")

## Classes: Organizing Complex Analysis Pipelines 🏗️

In [ ]:
class EEGProcessor:
    """A simple EEG processing class to demonstrate object-oriented concepts."""
    
    def __init__(self, sampling_rate=250, channels=None):
        self.sampling_rate = sampling_rate
        self.channels = channels or ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4']
        self.processed_data = None
        
    def load_data(self, data):
        """Load EEG data (channels × time_points)."""
        self.raw_data = np.array(data)
        print(f"Data loaded: {self.raw_data.shape} (channels × time_points)")
        
    def apply_bandpass_filter(self, low_freq=1, high_freq=40):
        """Apply a bandpass filter to remove noise."""
        if self.raw_data is None:
            raise ValueError("No data loaded! Use load_data() first.")
            
        # Design butterworth filter
        sos = signal.butter(4, [low_freq, high_freq], 
                           btype='bandpass', fs=self.sampling_rate, output='sos')
        
        # Apply filter to each channel
        self.processed_data = np.zeros_like(self.raw_data)
        for i, channel_data in enumerate(self.raw_data):
            self.processed_data[i] = signal.sosfilt(sos, channel_data)
            
        print(f"✅ Bandpass filter applied ({low_freq}-{high_freq} Hz)")
        
    def plot_channel(self, channel_idx=0, duration=2.0):
        """Plot raw vs processed data for a specific channel."""
        if self.raw_data is None:
            raise ValueError("No data loaded!")
            
        # Time axis
        n_samples = int(duration * self.sampling_rate)
        time_axis = np.linspace(0, duration, n_samples)
        
        plt.figure(figsize=(12, 6))
        
        # Plot raw data
        plt.subplot(2, 1, 1)
        plt.plot(time_axis, self.raw_data[channel_idx, :n_samples], 'b-', alpha=0.7)
        plt.title(f'Raw EEG - Channel {self.channels[channel_idx]}')
        plt.ylabel('Amplitude (μV)')
        plt.grid(True, alpha=0.3)
        
        # Plot processed data (if available)
        if self.processed_data is not None:
            plt.subplot(2, 1, 2)
            plt.plot(time_axis, self.processed_data[channel_idx, :n_samples], 'r-', alpha=0.7)
            plt.title(f'Filtered EEG - Channel {self.channels[channel_idx]}')
            plt.ylabel('Amplitude (μV)')
            plt.xlabel('Time (seconds)')
            plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Demo the class
processor = EEGProcessor(sampling_rate=250)

# Simulate some EEG data with noise
time_points = np.linspace(0, 10, 2500)  # 10 seconds at 250 Hz
n_channels = len(processor.channels)

# Create realistic EEG-like signals
fake_eeg = np.zeros((n_channels, len(time_points)))
for i in range(n_channels):
    # Alpha waves (10 Hz) + noise + some artifacts
    alpha_wave = 2 * np.sin(2 * np.pi * 10 * time_points + i * 0.5)
    theta_wave = 1 * np.sin(2 * np.pi * 6 * time_points + i * 0.3)
    noise = 0.5 * np.random.randn(len(time_points))
    artifacts = 10 * np.sin(2 * np.pi * 60 * time_points)  # 60 Hz line noise
    
    fake_eeg[i] = alpha_wave + theta_wave + noise + artifacts

# Process the data
processor.load_data(fake_eeg)
processor.apply_bandpass_filter(low_freq=1, high_freq=40)
processor.plot_channel(channel_idx=0, duration=3.0)

# 🔢 NumPy Fundamentals: The Heart of Scientific Computing

NumPy is the foundation of scientific computing in Python. In neuroscience, you'll use it for everything from basic array operations to complex signal processing.

## Array Creation and Basic Operations 🎯

In [ ]:
# Different ways to create arrays (common in neuroscience)

# 1. Time axis for signals
sampling_rate = 250  # Hz
duration = 2.0  # seconds
time_axis = np.linspace(0, duration, int(sampling_rate * duration))
print(f"Time axis: {time_axis[:5]}...{time_axis[-5:]}")
print(f"Shape: {time_axis.shape}")

# 2. Initialize arrays for data storage
n_channels = 64
n_timepoints = 1000
n_trials = 100

# Empty array for EEG data (channels × time × trials)
eeg_data = np.zeros((n_channels, n_timepoints, n_trials))
print(f"\nEEG data array shape: {eeg_data.shape}")

# Random data (useful for testing)
random_signals = np.random.randn(n_channels, n_timepoints)
print(f"Random signals shape: {random_signals.shape}")

# 3. Structured arrays for experiments
trial_conditions = np.array(['rest', 'task', 'rest', 'task'] * 25)
print(f"\nTrial conditions (first 10): {trial_conditions[:10]}")

# 4. Frequency arrays for spectral analysis
frequencies = np.fft.fftfreq(n_timepoints, 1/sampling_rate)
positive_freqs = frequencies[:n_timepoints//2]
print(f"\nFrequency range: {positive_freqs[0]:.1f} to {positive_freqs[-1]:.1f} Hz")

## Array Indexing and Slicing: Extracting What You Need 🎯

In neuroscience, you'll constantly need to extract specific time windows, channels, or trials.

In [ ]:
# Create sample EEG data (channels × time × trials)
np.random.seed(42)  # For reproducible results
n_channels, n_timepoints, n_trials = 10, 1000, 50
eeg_data = np.random.randn(n_channels, n_timepoints, n_trials)

# Add some structure to make it more realistic
time_axis = np.linspace(0, 4, n_timepoints)  # 4 seconds
for ch in range(n_channels):
    for trial in range(n_trials):
        # Add alpha waves (10 Hz) with some variability
        alpha_freq = 10 + np.random.normal(0, 1)
        eeg_data[ch, :, trial] += 2 * np.sin(2 * np.pi * alpha_freq * time_axis)

print(f"EEG data shape: {eeg_data.shape}")
print(f"Data type: {eeg_data.dtype}")
print(f"Memory usage: {eeg_data.nbytes / 1024:.1f} KB")

# Common indexing operations
print("\n=== Common Indexing Operations ===")

# 1. Select specific channels
frontal_channels = [0, 1, 2]  # Assuming these are frontal
frontal_data = eeg_data[frontal_channels, :, :]
print(f"1. Frontal channels data shape: {frontal_data.shape}")

# 2. Select time window (e.g., 1-3 seconds)
start_time, end_time = 1.0, 3.0
start_idx = int(start_time * (n_timepoints / 4))  # 4 seconds total
end_idx = int(end_time * (n_timepoints / 4))
time_window = eeg_data[:, start_idx:end_idx, :]
print(f"2. Time window (1-3s) shape: {time_window.shape}")

# 3. Select specific trials
task_trials = [1, 3, 5, 7, 9]  # Assuming these are task trials
task_data = eeg_data[:, :, task_trials]
print(f"3. Task trials data shape: {task_data.shape}")

# 4. Complex indexing: frontal channels, specific time window, task trials
# First select channels, then time window, then trials
subset = eeg_data[np.ix_(frontal_channels, range(start_idx, end_idx), task_trials)]
print(f"4. Complex subset shape: {subset.shape}")

# Alternative approach using step-by-step indexing
# subset_step1 = eeg_data[frontal_channels, :, :]  # Select channels
# subset_step2 = subset_step1[:, start_idx:end_idx, :]  # Select time window
# subset_final = subset_step2[:, :, task_trials]  # Select trials
# print(f"4. Complex subset shape (alternative): {subset_final.shape}")

# 5. Boolean indexing (very powerful!)
# Find time points where signal is above threshold
channel_0_trial_0 = eeg_data[0, :, 0]
high_amplitude_mask = np.abs(channel_0_trial_0) > 2
high_amplitude_points = channel_0_trial_0[high_amplitude_mask]
print(f"5. High amplitude points: {len(high_amplitude_points)} out of {n_timepoints}")

# 6. Advanced: condition-based selection
# Get trials where the mean amplitude is above median
trial_means = np.mean(eeg_data, axis=(0, 1))  # Mean across channels and time
high_activity_trials = trial_means > np.median(trial_means)
high_activity_data = eeg_data[:, :, high_activity_trials]
print(f"6. High activity trials: {high_activity_data.shape[2]} out of {n_trials}")

## Array Operations: The Power of Vectorization ⚡

Vectorized operations are much faster than Python loops. This is crucial when processing large neuroscience datasets.

In [ ]:
# 🚀 Performance & Optimization: Critical for Large Neuroscience Datasets

import time
import numpy as np
from functools import wraps

print("⚡ Performance Optimization for Neuroscience Computing")
print("=" * 55)

# Performance testing decorator
def time_it(func):
    """Decorator to time function execution"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"⏱️ {func.__name__}: {end_time - start_time:.4f} seconds")
        return result
    return wrapper

# Memory usage tracking
def track_memory(func):
    """Decorator to track memory usage"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        try:
            import psutil
            import os
            process = psutil.Process(os.getpid())
            mem_before = process.memory_info().rss / 1024**2  # MB
            
            result = func(*args, **kwargs)
            
            mem_after = process.memory_info().rss / 1024**2  # MB
            print(f"💾 {func.__name__}: {mem_after - mem_before:.1f} MB memory increase")
            return result
        except ImportError:
            return func(*args, **kwargs)
    return wrapper

print("\n1. 🐌 LOOP VS ⚡ VECTORIZED OPERATIONS")
print("=" * 45)

# Generate realistic EEG-like data
n_samples = 100000
eeg_signal = np.random.randn(n_samples) * 50  # 50 μV typical EEG amplitude

print(f"Testing with {n_samples:,} EEG samples...")

# Method 1: Python loop (slow)
@time_it
def apply_filter_loop(signal):
    """Apply a simple moving average filter using Python loops"""
    filtered = []
    window_size = 5
    
    for i in range(len(signal)):
        if i < window_size:
            filtered.append(signal[i])
        else:
            # Moving average
            avg = sum(signal[i-window_size:i]) / window_size
            filtered.append(avg)
    
    return np.array(filtered)

# Method 2: NumPy vectorized (fast)
@time_it
def apply_filter_vectorized(signal):
    """Apply moving average filter using NumPy convolution"""
    window_size = 5
    kernel = np.ones(window_size) / window_size
    # Use 'same' mode to maintain original length
    filtered = np.convolve(signal, kernel, mode='same')
    return filtered

# Method 3: SciPy optimized (fastest)
@time_it
def apply_filter_scipy(signal):
    """Apply moving average using SciPy's optimized functions"""
    from scipy import ndimage
    filtered = ndimage.uniform_filter1d(signal, size=5, mode='constant')
    return filtered

print("\n🏁 Speed Comparison:")
result_loop = apply_filter_loop(eeg_signal)
result_vectorized = apply_filter_vectorized(eeg_signal)
result_scipy = apply_filter_scipy(eeg_signal)

# Verify results are similar
print(f"\n🔍 Results validation:")
print(f"   Loop vs Vectorized max difference: {np.max(np.abs(result_loop - result_vectorized)):.6f}")
print(f"   Vectorized vs SciPy max difference: {np.max(np.abs(result_vectorized - result_scipy)):.6f}")

print("\n\n2. 💾 MEMORY OPTIMIZATION")
print("=" * 30)

@track_memory
@time_it
def memory_inefficient_analysis(n_channels=64, n_timepoints=10000, n_trials=100):
    """Demonstrate memory-inefficient approach"""
    
    # Create large dataset
    eeg_data = np.random.randn(n_channels, n_timepoints, n_trials) * 50
    
    # Inefficient: Create many intermediate arrays
    step1 = eeg_data ** 2  # Power
    step2 = np.sqrt(step1)  # Back to amplitude
    step3 = step1 + step2  # Combine
    step4 = np.mean(step3, axis=1)  # Average over time
    step5 = np.std(step4, axis=1)   # Standard deviation over trials
    
    return step5

@track_memory
@time_it
def memory_efficient_analysis(n_channels=64, n_timepoints=10000, n_trials=100):
    """Demonstrate memory-efficient approach"""
    
    # Create dataset
    eeg_data = np.random.randn(n_channels, n_timepoints, n_trials) * 50
    
    # Efficient: Chain operations without intermediate storage
    result = np.std(np.mean(eeg_data**2 + np.abs(eeg_data), axis=1), axis=1)
    
    return result

print("\n🧪 Memory Usage Comparison:")
print("Inefficient approach:")
result1 = memory_inefficient_analysis(32, 5000, 50)  # Smaller for demo

print("\nEfficient approach:")
result2 = memory_efficient_analysis(32, 5000, 50)

print(f"\nResults are similar: {np.allclose(result1, result2)}")

print("\n\n3. 🔄 PARALLEL PROCESSING")
print("=" * 25)

def process_single_channel(channel_data):
    """Process a single EEG channel (example: power spectral density)"""
    from scipy.signal import welch
    frequencies, psd = welch(channel_data, fs=250, nperseg=256)
    # Return alpha power (8-13 Hz)
    alpha_mask = (frequencies >= 8) & (frequencies <= 13)
    alpha_power = np.trapz(psd[alpha_mask], frequencies[alpha_mask])
    return alpha_power

@time_it
def sequential_processing(eeg_data):
    """Process channels sequentially"""
    results = []
    for channel in eeg_data:
        alpha_power = process_single_channel(channel)
        results.append(alpha_power)
    return np.array(results)

@time_it
def parallel_processing(eeg_data):
    """Process channels in parallel"""
    try:
        from multiprocessing import Pool
        import os
        
        # Use half the available cores to avoid overloading
        n_cores = max(1, os.cpu_count() // 2)
        
        with Pool(processes=n_cores) as pool:
            results = pool.map(process_single_channel, eeg_data)
        
        return np.array(results)
        
    except Exception as e:
        print(f"   ⚠️ Parallel processing failed: {e}")
        print("   📝 Falling back to sequential processing")
        return sequential_processing(eeg_data)

# Test with moderate-sized data
test_eeg = np.random.randn(32, 2500) * 50  # 32 channels, 10 seconds at 250 Hz

print("\n🏁 Processing Speed Comparison:")
print("Sequential processing:")
seq_result = sequential_processing(test_eeg)

print("Parallel processing:")
par_result = parallel_processing(test_eeg)

print(f"\nResults match: {np.allclose(seq_result, par_result)}")

print("\n\n4. 📊 PROFILING YOUR CODE")
print("=" * 30)

def profile_function(func, *args, **kwargs):
    """Simple profiling function"""
    import cProfile
    import pstats
    import io
    
    # Create a profiler
    profiler = cProfile.Profile()
    
    # Profile the function
    profiler.enable()
    result = func(*args, **kwargs)
    profiler.disable()
    
    # Get stats
    stats_stream = io.StringIO()
    ps = pstats.Stats(profiler, stream=stats_stream)
    ps.sort_stats('cumulative')
    ps.print_stats(10)  # Top 10 functions
    
    print("📈 Top time-consuming functions:")
    print(stats_stream.getvalue())
    
    return result

def complex_eeg_analysis(eeg_data):
    """A more complex analysis function to profile"""
    
    # Bandpass filtering simulation
    filtered = np.zeros_like(eeg_data)
    for i, channel in enumerate(eeg_data):
        # Simple high-pass filter (remove DC)
        filtered[i] = channel - np.mean(channel)
    
    # Compute power in different bands
    from scipy.signal import welch
    
    power_bands = {}
    bands = {'alpha': (8, 13), 'beta': (13, 30), 'theta': (4, 8)}
    
    for band_name, (low, high) in bands.items():
        band_powers = []
        for channel in filtered:
            freqs, psd = welch(channel, fs=250, nperseg=256)
            mask = (freqs >= low) & (freqs <= high)
            power = np.trapz(psd[mask], freqs[mask])
            band_powers.append(power)
        power_bands[band_name] = np.array(band_powers)
    
    # Correlation analysis
    correlation_matrix = np.corrcoef(filtered)
    
    # Feature extraction
    features = np.concatenate([
        power_bands['alpha'],
        power_bands['beta'],
        power_bands['theta'],
        np.diag(correlation_matrix)
    ])
    
    return features

# Profile the complex analysis
print("🔍 Profiling complex EEG analysis...")
sample_data = np.random.randn(16, 1250) * 50  # 16 channels, 5 seconds
profiled_result = profile_function(complex_eeg_analysis, sample_data)

print("\n\n💡 OPTIMIZATION STRATEGIES")
print("=" * 35)

optimization_tips = [
    "✅ Use NumPy/SciPy built-in functions instead of pure Python loops",
    "✅ Minimize intermediate array creation with chained operations", 
    "✅ Choose appropriate data types (float32 vs float64)",
    "✅ Use in-place operations when possible (+=, *=, etc.)",
    "✅ Leverage parallel processing for embarrassingly parallel tasks",
    "✅ Profile your code to identify actual bottlenecks",
    "✅ Consider using numba for JIT compilation of critical functions",
    "✅ Use memory mapping for very large datasets",
    "✅ Batch operations to reduce function call overhead",
    "✅ Cache expensive computations when reused"
]

for tip in optimization_tips:
    print(f"   {tip}")

print("\n📚 Performance Benchmarks Saved:")
print("   - Loop vs vectorized speedup factor")
print("   - Memory usage differences")  
print("   - Parallel processing overhead")
print("   - Function profiling results")

print("\n🎯 Key Takeaway: Always measure before optimizing!")
print("   Use vectorized operations for 10-100x speedups in neuroscience computing.")

## Mathematical Operations for Neuroscience 🧮

Let's explore the mathematical operations you'll use most frequently in brain signal analysis.

In [ ]:
# Create sample multi-channel EEG data
np.random.seed(123)
n_channels, n_timepoints, n_trials = 8, 500, 20
eeg_data = np.random.randn(n_channels, n_timepoints, n_trials)

# Add realistic signal structure
time_axis = np.linspace(0, 2, n_timepoints)  # 2 seconds
for ch in range(n_channels):
    for trial in range(n_trials):
        # Add alpha rhythm (10 Hz) and some theta (6 Hz)
        alpha = 3 * np.sin(2 * np.pi * 10 * time_axis + ch * 0.2)
        theta = 1.5 * np.sin(2 * np.pi * 6 * time_axis + ch * 0.1)
        eeg_data[ch, :, trial] += alpha + theta

print(f"Sample EEG data shape: {eeg_data.shape}")
print(f"Data range: {eeg_data.min():.2f} to {eeg_data.max():.2f}")

# === Statistical Operations ===
print("\n=== Statistical Operations ===")

# 1. Basic statistics across different axes
trial_averages = np.mean(eeg_data, axis=2)  # Average across trials
channel_averages = np.mean(eeg_data, axis=0)  # Average across channels
time_averages = np.mean(eeg_data, axis=1)  # Average across time

print(f"Trial averages shape: {trial_averages.shape}")
print(f"Channel averages shape: {channel_averages.shape}")
print(f"Time averages shape: {time_averages.shape}")

# 2. Standard deviation and variance
trial_std = np.std(eeg_data, axis=2)
print(f"\nStandard deviation across trials: {trial_std.mean():.3f}")

# 3. Percentiles (useful for outlier detection)
percentiles = np.percentile(eeg_data, [5, 25, 50, 75, 95])
print(f"Data percentiles: {percentiles}")

# === Signal Processing Operations ===
print("\n=== Signal Processing Operations ===")

# 1. Z-score normalization (common preprocessing step)
eeg_normalized = (eeg_data - np.mean(eeg_data, axis=1, keepdims=True)) / np.std(eeg_data, axis=1, keepdims=True)
print(f"Normalized data mean: {np.mean(eeg_normalized):.6f}")
print(f"Normalized data std: {np.std(eeg_normalized):.6f}")

# 2. Baseline correction (subtract pre-stimulus period)
baseline_period = slice(0, 50)  # First 50 time points
baseline_mean = np.mean(eeg_data[:, baseline_period, :], axis=1, keepdims=True)
eeg_baseline_corrected = eeg_data - baseline_mean
print(f"Baseline corrected data range: {eeg_baseline_corrected.min():.2f} to {eeg_baseline_corrected.max():.2f}")

# 3. Root Mean Square (RMS) - measure of signal power
rms_values = np.sqrt(np.mean(eeg_data**2, axis=1))
print(f"RMS values shape: {rms_values.shape}")
print(f"Average RMS across channels: {np.mean(rms_values):.3f}")

# === Advanced Operations ===
print("\n=== Advanced Operations ===")

# 1. Correlation between channels
# Take first trial, correlate channels across time
channel_correlations = np.corrcoef(eeg_data[:, :, 0])
print(f"Channel correlation matrix shape: {channel_correlations.shape}")
print(f"Average correlation: {np.mean(channel_correlations[np.triu_indices(n_channels, k=1)]):.3f}")

# 2. Covariance matrix
cov_matrix = np.cov(eeg_data[:, :, 0])
print(f"Covariance matrix shape: {cov_matrix.shape}")

# 3. Principal Component Analysis (PCA) preparation
# Center the data
data_centered = eeg_data - np.mean(eeg_data, axis=1, keepdims=True)
# Compute covariance matrix
cov_temporal = np.cov(data_centered[:, :, 0].T)  # Time x Time covariance
eigenvalues, eigenvectors = np.linalg.eig(cov_temporal)
print(f"Top 5 eigenvalues: {np.sort(eigenvalues)[-5:][::-1]}")

# 4. Sliding window analysis
window_size = 50
step_size = 10
windows = []
for start in range(0, n_timepoints - window_size, step_size):
    window_data = eeg_data[:, start:start+window_size, :]
    window_mean = np.mean(window_data, axis=1)  # Average across time in window
    windows.append(window_mean)

windowed_analysis = np.array(windows)
print(f"Windowed analysis shape: {windowed_analysis.shape}")
print(f"Number of windows: {len(windows)}")

## Linear Algebra for Neural Networks 🧠

Understanding matrix operations is crucial for neural networks and many signal processing techniques.

In [ ]:
# === Matrix Operations Fundamental to Neural Networks ===
print("=== Matrix Operations for Neural Networks ===")

# 1. Basic matrix multiplication (the heart of neural networks)
# Simulate a simple neural network layer
input_features = 64  # e.g., 64 EEG channels
hidden_units = 32
batch_size = 10

# Input data (batch_size × input_features)
X = np.random.randn(batch_size, input_features)
print(f"Input X shape: {X.shape}")

# Weight matrix (input_features × hidden_units)
W = np.random.randn(input_features, hidden_units) * 0.1  # Small random weights
print(f"Weight W shape: {W.shape}")

# Bias vector (hidden_units,)
b = np.zeros(hidden_units)
print(f"Bias b shape: {b.shape}")

# Forward pass: linear transformation
z = np.dot(X, W) + b  # or X @ W + b
print(f"Linear output z shape: {z.shape}")

# Activation function (ReLU)
a = np.maximum(0, z)
print(f"Activated output a shape: {a.shape}")
print(f"Activated units: {np.sum(a > 0)} out of {a.size}")

# 2. Batch operations (processing multiple samples simultaneously)
print("\n=== Batch Operations ===")

# Simulate processing multiple EEG epochs
n_epochs = 100
n_channels = 10
n_timepoints = 250

# Create batch of EEG data
eeg_batch = np.random.randn(n_epochs, n_channels, n_timepoints)
print(f"EEG batch shape: {eeg_batch.shape}")

# Flatten each epoch for neural network input
eeg_flattened = eeg_batch.reshape(n_epochs, -1)
print(f"Flattened EEG shape: {eeg_flattened.shape}")

# Apply transformation to entire batch
feature_weights = np.random.randn(n_channels * n_timepoints, 50)
features = np.dot(eeg_flattened, feature_weights)
print(f"Extracted features shape: {features.shape}")

# 3. Covariance and correlation matrices (important for connectivity analysis)
print("\n=== Covariance and Correlation ===")

# Simulate multi-channel EEG data
n_channels = 5
n_timepoints = 1000
np.random.seed(42)

# Create correlated channels (simulate brain connectivity)
base_signal = np.random.randn(n_timepoints)
channels = np.zeros((n_channels, n_timepoints))

for i in range(n_channels):
    # Each channel is base signal + independent noise
    channels[i] = 0.7 * base_signal + 0.3 * np.random.randn(n_timepoints)

# Compute correlation matrix
correlation_matrix = np.corrcoef(channels)
print(f"Correlation matrix shape: {correlation_matrix.shape}")
print(f"Correlation matrix:")
print(correlation_matrix)

# Compute covariance matrix
covariance_matrix = np.cov(channels)
print(f"\nCovariance matrix shape: {covariance_matrix.shape}")

# 4. Eigendecomposition (used in PCA, ICA, etc.)
print("\n=== Eigendecomposition ===")

eigenvalues, eigenvectors = np.linalg.eig(correlation_matrix)
print(f"Eigenvalues: {eigenvalues}")
print(f"Eigenvectors shape: {eigenvectors.shape}")

# Sort by eigenvalue magnitude
idx = np.argsort(eigenvalues)[::-1]
eigenvalues_sorted = eigenvalues[idx]
eigenvectors_sorted = eigenvectors[:, idx]

print(f"Sorted eigenvalues: {eigenvalues_sorted}")
print(f"Explained variance ratio: {eigenvalues_sorted / np.sum(eigenvalues_sorted)}")

# 5. Matrix norms (useful for regularization)
print("\n=== Matrix Norms ===")

sample_matrix = np.random.randn(5, 5)
print(f"Frobenius norm: {np.linalg.norm(sample_matrix, 'fro'):.3f}")
print(f"Nuclear norm: {np.linalg.norm(sample_matrix, 'nuc'):.3f}")
print(f"Spectral norm: {np.linalg.norm(sample_matrix, 2):.3f}")

# 6. Solving linear systems (used in many algorithms)
print("\n=== Solving Linear Systems ===")

# Example: least squares solution
A = np.random.randn(100, 10)  # Design matrix
x_true = np.random.randn(10)  # True parameters
y = A @ x_true + 0.1 * np.random.randn(100)  # Noisy observations

# Solve least squares: x = (A^T A)^(-1) A^T y
x_estimated = np.linalg.solve(A.T @ A, A.T @ y)
print(f"True parameters: {x_true[:5]}")
print(f"Estimated parameters: {x_estimated[:5]}")
print(f"Estimation error: {np.linalg.norm(x_true - x_estimated):.6f}")

# 🧬 Real-World Example: EEG Signal Analysis

Let's put everything together with a realistic neuroscience example: analyzing EEG signals to detect different brain states.

In [ ]:
# 🏥 Real-World Clinical Applications & Case Studies

print("🌍 Real-World Applications of Python & NumPy in Neuroscience")
print("=" * 60)

import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import time

print("🎯 Learning Objective: Connect programming skills to actual clinical practice")
print("📊 See how the techniques you've learned solve real neuroscience problems")

# Case Study 1: Seizure Detection Algorithm
print("\n\n🚨 CASE STUDY 1: Epileptic Seizure Detection")
print("=" * 50)

def simulate_seizure_detection():
    """Simulate a basic seizure detection algorithm using NumPy"""
    
    print("🏥 Clinical Context:")
    print("   - Epilepsy affects 65 million people worldwide")
    print("   - Early seizure detection can trigger interventions")
    print("   - Automated detection reduces monitoring burden")
    print("   - Real-time processing requires efficient algorithms")
    
    # Simulate EEG data with seizure activity
    sampling_rate = 250  # Hz
    duration = 60  # seconds (1 minute recording)
    n_channels = 19  # Standard 10-20 EEG montage
    
    time_axis = np.linspace(0, duration, sampling_rate * duration)
    
    # Normal background EEG (alpha + noise)
    normal_eeg = np.zeros((n_channels, len(time_axis)))
    for ch in range(n_channels):
        # Alpha waves with some variation
        alpha = 2 * np.sin(2 * np.pi * (10 + np.random.normal(0, 0.5)) * time_axis)
        noise = 1.5 * np.random.randn(len(time_axis))
        normal_eeg[ch] = alpha + noise
    
    # Add seizure activity (high amplitude, synchronized oscillations)
    seizure_start = 30  # seconds
    seizure_duration = 15  # seconds
    seizure_start_idx = int(seizure_start * sampling_rate)
    seizure_end_idx = int((seizure_start + seizure_duration) * sampling_rate)
    
    seizure_eeg = normal_eeg.copy()
    
    # Create seizure pattern - high frequency, high amplitude, synchronized
    seizure_time = time_axis[seizure_start_idx:seizure_end_idx]
    seizure_freq = 15  # Hz (typical seizure frequency)
    
    for ch in range(n_channels):
        # Synchronized seizure activity across channels
        phase_shift = ch * 0.1  # Small phase differences
        seizure_pattern = 8 * np.sin(2 * np.pi * seizure_freq * seizure_time + phase_shift)
        
        # Add harmonics for realism
        seizure_pattern += 4 * np.sin(2 * np.pi * 2 * seizure_freq * seizure_time + phase_shift)
        
        seizure_eeg[ch, seizure_start_idx:seizure_end_idx] += seizure_pattern
    
    return seizure_eeg, time_axis, (seizure_start, seizure_start + seizure_duration)

def seizure_detection_algorithm(eeg_data, sampling_rate=250, window_size=5):
    """
    Simple seizure detection based on:
    1. High amplitude detection
    2. Spectral power analysis  
    3. Cross-channel synchronization
    """
    
    n_channels, n_timepoints = eeg_data.shape
    window_samples = int(window_size * sampling_rate)
    n_windows = n_timepoints // window_samples
    
    detection_scores = []
    
    for w in range(n_windows):
        start_idx = w * window_samples
        end_idx = start_idx + window_samples
        window_data = eeg_data[:, start_idx:end_idx]
        
        # Feature 1: High amplitude detection
        amplitude_score = np.mean(np.abs(window_data))
        
        # Feature 2: Spectral power in seizure band (10-25 Hz)
        freqs, psd = signal.welch(window_data.mean(axis=0), fs=sampling_rate, nperseg=128)
        seizure_band_mask = (freqs >= 10) & (freqs <= 25)
        spectral_score = np.trapz(psd[seizure_band_mask], freqs[seizure_band_mask])
        
        # Feature 3: Cross-channel synchronization
        correlations = np.corrcoef(window_data)
        sync_score = np.mean(correlations[np.triu_indices_from(correlations, k=1)])
        
        # Combine features (simple weighted sum)
        combined_score = (amplitude_score * 0.4 + 
                         spectral_score * 0.4 + 
                         sync_score * 0.2)
        
        detection_scores.append(combined_score)
    
    return np.array(detection_scores)

# Run seizure detection simulation
seizure_eeg, time_axis, seizure_period = simulate_seizure_detection()
detection_scores = seizure_detection_algorithm(seizure_eeg)

# Visualize results
fig, axes = plt.subplots(3, 1, figsize=(15, 10))

# Plot raw EEG (selected channels)
selected_channels = [0, 5, 10]
for i, ch in enumerate(selected_channels):
    axes[0].plot(time_axis, seizure_eeg[ch] + i*20, label=f'Channel {ch}', alpha=0.8)

axes[0].axvspan(seizure_period[0], seizure_period[1], alpha=0.3, color='red', label='Actual Seizure')
axes[0].set_ylabel('Amplitude (μV)')
axes[0].set_title('Simulated EEG with Seizure Activity')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot detection scores
window_times = np.linspace(0, 60, len(detection_scores))
axes[1].plot(window_times, detection_scores, 'b-', linewidth=2, label='Detection Score')
axes[1].axhline(y=np.mean(detection_scores) + 2*np.std(detection_scores), 
                color='red', linestyle='--', label='Detection Threshold')
axes[1].axvspan(seizure_period[0], seizure_period[1], alpha=0.3, color='red')
axes[1].set_ylabel('Detection Score')
axes[1].set_title('Seizure Detection Algorithm Output')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Plot spectral analysis
freqs, psd_normal = signal.welch(seizure_eeg[:, :7500].mean(axis=0), fs=250, nperseg=512)
freqs, psd_seizure = signal.welch(seizure_eeg[:, 7500:11250].mean(axis=0), fs=250, nperseg=512)

axes[2].semilogy(freqs, psd_normal, label='Normal Period', alpha=0.8)
axes[2].semilogy(freqs, psd_seizure, label='Seizure Period', alpha=0.8)
axes[2].axvspan(10, 25, alpha=0.2, color='red', label='Seizure Band')
axes[2].set_xlim(0, 40)
axes[2].set_xlabel('Frequency (Hz)')
axes[2].set_ylabel('Power Spectral Density')
axes[2].set_title('Spectral Analysis: Normal vs Seizure')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate detection performance
threshold = np.mean(detection_scores) + 2*np.std(detection_scores)
detected_windows = detection_scores > threshold
seizure_windows = (window_times >= seizure_period[0]) & (window_times <= seizure_period[1])

true_positives = np.sum(detected_windows & seizure_windows)
false_positives = np.sum(detected_windows & ~seizure_windows)
true_negatives = np.sum(~detected_windows & ~seizure_windows)
false_negatives = np.sum(~detected_windows & seizure_windows)

sensitivity = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
specificity = true_negatives / (true_negatives + false_positives) if (true_negatives + false_positives) > 0 else 0

print(f"\n📊 Detection Performance:")
print(f"   Sensitivity (True Positive Rate): {sensitivity:.2f}")
print(f"   Specificity (True Negative Rate): {specificity:.2f}")
print(f"   True Positives: {true_positives}, False Positives: {false_positives}")
print(f"   True Negatives: {true_negatives}, False Negatives: {false_negatives}")

print("\n🏥 Clinical Impact:")
print("   ✅ Automated seizure detection enables:")
print("      • 24/7 monitoring without constant human oversight")
print("      • Immediate alerts for medical intervention")
print("      • Objective seizure counting for treatment evaluation")
print("      • Early warning systems for seizure prediction")

simulate_seizure_detection()

# Case Study 2: Sleep Stage Classification
print("\n\n😴 CASE STUDY 2: Sleep Stage Classification")
print("=" * 50)

def simulate_sleep_analysis():
    """Simulate sleep stage classification using EEG features"""
    
    print("🏥 Clinical Context:")
    print("   - Sleep disorders affect 50-70 million US adults")
    print("   - Polysomnography requires manual scoring (expensive, time-consuming)")
    print("   - Automated staging improves diagnosis accessibility")
    print("   - Different sleep stages have distinct EEG signatures")
    
    # Sleep stages and their characteristics
    sleep_stages = {
        'Wake': {'freq_bands': {'beta': 2.0, 'alpha': 1.5, 'theta': 0.5}, 'amplitude': 30},
        'N1': {'freq_bands': {'theta': 2.0, 'alpha': 1.0, 'beta': 0.5}, 'amplitude': 40},
        'N2': {'freq_bands': {'theta': 1.5, 'sigma': 2.5, 'delta': 1.0}, 'amplitude': 50},  # Sleep spindles
        'N3': {'freq_bands': {'delta': 4.0, 'theta': 0.5}, 'amplitude': 80},  # Deep sleep
        'REM': {'freq_bands': {'theta': 2.0, 'alpha': 1.0, 'beta': 1.5}, 'amplitude': 35}  # Similar to wake
    }
    
    frequency_bands = {
        'delta': 2,    # 0.5-4 Hz
        'theta': 6,    # 4-8 Hz  
        'alpha': 10,   # 8-13 Hz
        'sigma': 12,   # 11-15 Hz (sleep spindles)
        'beta': 20     # 13-30 Hz
    }
    
    # Simulate 8 hours of sleep data
    sampling_rate = 100  # Hz (lower for sleep studies)
    epoch_duration = 30  # seconds per epoch (standard)
    n_epochs = 8 * 60 * 60 // epoch_duration  # 960 epochs for 8 hours
    epoch_samples = sampling_rate * epoch_duration
    
    # Realistic sleep progression
    stage_sequence = (
        ['Wake'] * 10 +           # 5 minutes wake
        ['N1'] * 4 +              # 2 minutes N1
        ['N2'] * 20 +             # 10 minutes N2
        ['N3'] * 40 +             # 20 minutes N3 (deep sleep)
        ['N2'] * 15 +             # Back to N2
        ['REM'] * 15 +            # First REM period
        (['N2'] * 30 + ['N3'] * 20 + ['N2'] * 10 + ['REM'] * 20) * 6  # Sleep cycles
    )
    
    # Pad or truncate to match n_epochs
    if len(stage_sequence) < n_epochs:
        stage_sequence.extend(['N2'] * (n_epochs - len(stage_sequence)))
    else:
        stage_sequence = stage_sequence[:n_epochs]
    
    # Generate EEG data for each epoch
    simulated_eeg = []
    true_stages = []
    
    for epoch_idx, stage in enumerate(stage_sequence):
        stage_info = sleep_stages[stage]
        
        # Generate epoch data
        epoch_data = np.zeros(epoch_samples)
        
        # Add frequency components
        time_epoch = np.linspace(0, epoch_duration, epoch_samples)
        
        for band, amplitude in stage_info['freq_bands'].items():
            freq = frequency_bands[band]
            epoch_data += amplitude * np.sin(2 * np.pi * freq * time_epoch + 
                                           np.random.uniform(0, 2*np.pi))
        
        # Add noise
        noise_level = stage_info['amplitude'] * 0.3
        epoch_data += noise_level * np.random.randn(epoch_samples)
        
        simulated_eeg.append(epoch_data)
        true_stages.append(stage)
    
    return np.array(simulated_eeg), true_stages

def extract_sleep_features(eeg_epochs, sampling_rate=100):
    """Extract features for sleep stage classification"""
    
    features = []
    
    for epoch in eeg_epochs:
        # Compute power spectral density
        freqs, psd = signal.welch(epoch, fs=sampling_rate, nperseg=sampling_rate*4)
        
        # Extract band powers
        band_powers = {}
        bands = {
            'delta': (0.5, 4),
            'theta': (4, 8),
            'alpha': (8, 13),
            'sigma': (11, 15),  # Sleep spindles
            'beta': (13, 30)
        }
        
        for band_name, (low, high) in bands.items():
            mask = (freqs >= low) & (freqs <= high)
            band_powers[band_name] = np.trapz(psd[mask], freqs[mask])
        
        # Additional features
        total_power = sum(band_powers.values())
        relative_powers = {band: power/total_power for band, power in band_powers.items()}
        
        # Combine features
        epoch_features = [
            relative_powers['delta'],
            relative_powers['theta'], 
            relative_powers['alpha'],
            relative_powers['sigma'],
            relative_powers['beta'],
            band_powers['delta'] / band_powers['alpha'],  # Delta/Alpha ratio
            np.var(epoch),  # Signal variance
            np.mean(np.abs(epoch))  # Mean absolute amplitude
        ]
        
        features.append(epoch_features)
    
    return np.array(features)

def classify_sleep_stages(features, true_stages):
    """Simple rule-based sleep stage classification"""
    
    predicted_stages = []
    
    for feature_vector in features:
        delta_rel, theta_rel, alpha_rel, sigma_rel, beta_rel, delta_alpha_ratio, variance, amplitude = feature_vector
        
        # Simple decision rules (in practice, use machine learning)
        if delta_rel > 0.5 and amplitude > 50:
            stage = 'N3'  # Deep sleep: high delta
        elif sigma_rel > 0.15 and delta_alpha_ratio > 2:
            stage = 'N2'  # Sleep spindles present
        elif theta_rel > 0.3 and alpha_rel < 0.2:
            stage = 'N1'  # Transitional sleep
        elif beta_rel > 0.2 and alpha_rel > 0.2 and amplitude < 40:
            if theta_rel > 0.25:
                stage = 'REM'  # REM: mixed frequencies, low amplitude
            else:
                stage = 'Wake'  # Awake: beta and alpha present
        else:
            stage = 'N2'  # Default to N2
        
        predicted_stages.append(stage)
    
    return predicted_stages

# Run sleep analysis simulation
print("🔄 Generating 8 hours of simulated sleep EEG...")
sleep_eeg, true_stages = simulate_sleep_analysis()

print("🔄 Extracting sleep features...")
sleep_features = extract_sleep_features(sleep_eeg)

print("🔄 Classifying sleep stages...")
predicted_stages = classify_sleep_stages(sleep_features, true_stages)

# Calculate accuracy
stage_names = ['Wake', 'N1', 'N2', 'N3', 'REM']
accuracy = np.mean([true == pred for true, pred in zip(true_stages, predicted_stages)])

print(f"\n📊 Classification Results:")
print(f"   Overall Accuracy: {accuracy:.2f}")

# Confusion matrix
from collections import defaultdict
confusion_matrix = defaultdict(lambda: defaultdict(int))

for true, pred in zip(true_stages, predicted_stages):
    confusion_matrix[true][pred] += 1

print(f"\n📋 Confusion Matrix:")
print(f"{'True\\Pred':<8}", end="")
for stage in stage_names:
    print(f"{stage:>6}", end="")
print()

for true_stage in stage_names:
    print(f"{true_stage:<8}", end="")
    for pred_stage in stage_names:
        count = confusion_matrix[true_stage][pred_stage]
        print(f"{count:>6}", end="")
    print()

# Visualize hypnogram
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 8))

# True hypnogram
stage_to_num = {'Wake': 4, 'REM': 3, 'N1': 2, 'N2': 1, 'N3': 0}
true_numeric = [stage_to_num[stage] for stage in true_stages]
pred_numeric = [stage_to_num[stage] for stage in predicted_stages]

time_hours = np.arange(len(true_stages)) * 0.5 / 60  # Convert epochs to hours

ax1.plot(time_hours, true_numeric, 'b-', linewidth=2, label='True Stages')
ax1.set_ylabel('Sleep Stage')
ax1.set_title('True Sleep Hypnogram')
ax1.set_yticks(list(stage_to_num.values()))
ax1.set_yticklabels(list(stage_to_num.keys()))
ax1.grid(True, alpha=0.3)
ax1.set_xlim(0, 8)

ax2.plot(time_hours, pred_numeric, 'r-', linewidth=2, label='Predicted Stages')
ax2.set_xlabel('Time (hours)')
ax2.set_ylabel('Sleep Stage')
ax2.set_title('Predicted Sleep Hypnogram')
ax2.set_yticks(list(stage_to_num.values()))
ax2.set_yticklabels(list(stage_to_num.keys()))
ax2.grid(True, alpha=0.3)
ax2.set_xlim(0, 8)

plt.tight_layout()
plt.show()

print(f"\n🏥 Clinical Impact:")
print(f"   ✅ Automated sleep staging enables:")
print(f"      • Faster sleep study analysis (minutes vs hours)")
print(f"      • Consistent scoring across different technicians")
print(f"      • Large-scale sleep research studies")
print(f"      • Home sleep monitoring devices")
print(f"      • Early detection of sleep disorders")

simulate_sleep_analysis()

print("\n\n🧠 CASE STUDY 3: Brain-Computer Interface (BCI)")
print("=" * 55)

def simulate_motor_imagery_bci():
    """Simulate a motor imagery BCI using NumPy for feature extraction"""
    
    print("🏥 Clinical Context:")
    print("   - BCIs help paralyzed patients control devices with thoughts")
    print("   - Motor imagery creates detectable EEG patterns")
    print("   - Real-time classification enables device control")
    print("   - NumPy enables fast feature computation for real-time use")
    
    # Simulate motor imagery experiment
    sampling_rate = 250  # Hz
    trial_duration = 4    # seconds
    n_trials_per_class = 40
    n_channels = 22  # Motor cortex area
    
    # Motor imagery affects specific frequency bands
    mu_band = (8, 12)      # Mu rhythm suppression
    beta_band = (18, 26)   # Beta rhythm changes
    
    trials_left = []
    trials_right = []
    
    # Generate left hand imagery trials
    for trial in range(n_trials_per_class):
        time_axis = np.linspace(0, trial_duration, sampling_rate * trial_duration)
        trial_data = np.zeros((n_channels, len(time_axis)))
        
        for ch in range(n_channels):
            # Base EEG
            base_signal = 2 * np.random.randn(len(time_axis))
            
            # Motor imagery affects contralateral channels more
            if ch < n_channels // 2:  # Right hemisphere (left hand)
                # Mu rhythm suppression (Event-Related Desynchronization)
                mu_suppression = -1.5 * np.sin(2 * np.pi * 10 * time_axis)
                beta_enhancement = 1.0 * np.sin(2 * np.pi * 22 * time_axis)
            else:  # Left hemisphere
                mu_suppression = -0.5 * np.sin(2 * np.pi * 10 * time_axis)
                beta_enhancement = 0.3 * np.sin(2 * np.pi * 22 * time_axis)
            
            trial_data[ch] = base_signal + mu_suppression + beta_enhancement
        
        trials_left.append(trial_data)
    
    # Generate right hand imagery trials
    for trial in range(n_trials_per_class):
        time_axis = np.linspace(0, trial_duration, sampling_rate * trial_duration)
        trial_data = np.zeros((n_channels, len(time_axis)))
        
        for ch in range(n_channels):
            base_signal = 2 * np.random.randn(len(time_axis))
            
            if ch >= n_channels // 2:  # Left hemisphere (right hand)
                mu_suppression = -1.5 * np.sin(2 * np.pi * 10 * time_axis)
                beta_enhancement = 1.0 * np.sin(2 * np.pi * 22 * time_axis)
            else:  # Right hemisphere
                mu_suppression = -0.5 * np.sin(2 * np.pi * 10 * time_axis)
                beta_enhancement = 0.3 * np.sin(2 * np.pi * 22 * time_axis)
            
            trial_data[ch] = base_signal + mu_suppression + beta_enhancement
        
        trials_right.append(trial_data)
    
    return trials_left, trials_right

def extract_bci_features(trials, sampling_rate=250):
    """Extract Common Spatial Patterns (CSP) features for BCI"""
    
    features = []
    
    for trial in trials:
        # Simple power spectral features (in practice, use CSP)
        trial_features = []
        
        for ch in range(trial.shape[0]):
            # Compute power in mu and beta bands
            freqs, psd = signal.welch(trial[ch], fs=sampling_rate, nperseg=256)
            
            # Mu band power
            mu_mask = (freqs >= 8) & (freqs <= 12)
            mu_power = np.trapz(psd[mu_mask], freqs[mu_mask])
            
            # Beta band power  
            beta_mask = (freqs >= 18) & (freqs <= 26)
            beta_power = np.trapz(psd[beta_mask], freqs[beta_mask])
            
            trial_features.extend([mu_power, beta_power])
        
        features.append(trial_features)
    
    return np.array(features)

def classify_motor_imagery(features_left, features_right):
    """Simple classification using feature differences"""
    
    # Combine data
    X = np.vstack([features_left, features_right])
    y = np.hstack([np.zeros(len(features_left)), np.ones(len(features_right))])
    
    # Simple classification: compare hemisphere differences
    left_hemisphere_features = X[:, :X.shape[1]//2]
    right_hemisphere_features = X[:, X.shape[1]//2:]
    
    # Laterality index
    hemisphere_diff = np.mean(left_hemisphere_features, axis=1) - np.mean(right_hemisphere_features, axis=1)
    
    # Threshold-based classification
    threshold = np.median(hemisphere_diff)
    predictions = (hemisphere_diff > threshold).astype(int)
    
    accuracy = np.mean(predictions == y)
    
    return predictions, accuracy, hemisphere_diff

# Run BCI simulation
print("🔄 Simulating motor imagery BCI experiment...")
trials_left, trials_right = simulate_motor_imagery_bci()

print("🔄 Extracting BCI features...")
features_left = extract_bci_features(trials_left)
features_right = extract_bci_features(trials_right)

print("🔄 Training classifier...")
predictions, accuracy, laterality_scores = classify_motor_imagery(features_left, features_right)

print(f"\n📊 BCI Performance:")
print(f"   Classification Accuracy: {accuracy:.2f}")
print(f"   Chance Level: 0.50")
print(f"   Performance Above Chance: {accuracy > 0.6}")

# Visualize laterality patterns
plt.figure(figsize=(12, 6))

# Plot laterality scores
all_labels = np.hstack([np.zeros(len(features_left)), np.ones(len(features_right))])
colors = ['blue' if label == 0 else 'red' for label in all_labels]
labels = ['Left Hand' if label == 0 else 'Right Hand' for label in all_labels]

plt.scatter(range(len(laterality_scores)), laterality_scores, c=colors, alpha=0.6)
plt.axhline(y=np.median(laterality_scores), color='black', linestyle='--', label='Decision Threshold')
plt.xlabel('Trial Number')
plt.ylabel('Laterality Score (L-R Hemisphere Difference)')
plt.title('Motor Imagery BCI: Hemispheric Lateralization')
plt.legend(['Left Hand Imagery', 'Right Hand Imagery', 'Decision Threshold'])
plt.grid(True, alpha=0.3)
plt.show()

print(f"\n🏥 Clinical Impact:")
print(f"   ✅ Motor imagery BCIs enable:")
print(f"      • Wheelchair control for paralyzed patients")
print(f"      • Computer cursor control via thoughts")
print(f"      • Robotic arm control for amputees")
print(f"      • Communication devices for locked-in syndrome")
print(f"      • Stroke rehabilitation through neurofeedback")

simulate_motor_imagery_bci()

print("\n\n🌟 KEY TAKEAWAYS: NumPy in Clinical Practice")
print("=" * 55)

key_takeaways = [
    "🔬 Array operations enable real-time signal processing",
    "📊 Vectorized computations make clinical algorithms feasible", 
    "🏥 Simple statistical methods solve complex medical problems",
    "⚡ Performance optimization is critical for real-time applications",
    "🧠 Mathematical concepts translate directly to patient care",
    "📈 Feature extraction from signals enables automated diagnosis",
    "🤖 Automated analysis improves accuracy and reduces costs",
    "🎯 Pattern recognition helps detect subtle neurological changes"
]

for takeaway in key_takeaways:
    print(f"   {takeaway}")

print(f"\n💡 Professional Development:")
print(f"   • These skills are directly applicable in clinical research")
print(f"   • Understanding the math makes you a better collaborator")
print(f"   • Performance optimization skills are valued in medical device companies")
print(f"   • Signal processing knowledge applies to many biosignals (ECG, EMG, fMRI)")

print(f"\n🚀 Next Steps:")
print(f"   • Learn machine learning for more sophisticated classification")
print(f"   • Study digital signal processing for advanced filtering")
print(f"   • Explore deep learning for automatic feature discovery")
print(f"   • Practice with real datasets from public repositories")

In [ ]:
# === Exploratory Data Analysis ===
print("=== Exploratory Data Analysis ===")

# 1. Visualize sample trials
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot sample eyes-closed trial
axes[0, 0].plot(time_axis[:1000], eeg_data[6, :1000, 0])  # P3 channel, first 4 seconds
axes[0, 0].set_title('Eyes Closed - P3 Channel')
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('Amplitude (μV)')
axes[0, 0].grid(True, alpha=0.3)

# Plot sample eyes-open trial
axes[0, 1].plot(time_axis[:1000], eeg_data[6, :1000, 35])  # P3 channel, eyes-open trial
axes[0, 1].set_title('Eyes Open - P3 Channel')
axes[0, 1].set_xlabel('Time (s)')
axes[0, 1].set_ylabel('Amplitude (μV)')
axes[0, 1].grid(True, alpha=0.3)

# 2. Power spectral density comparison
from scipy.signal import welch

# Compute PSD for both conditions
freqs, psd_closed = welch(eeg_data[6, :, :30].mean(axis=1), sampling_rate, nperseg=sampling_rate*2)
freqs, psd_open = welch(eeg_data[6, :, 30:].mean(axis=1), sampling_rate, nperseg=sampling_rate*2)

axes[1, 0].semilogy(freqs, psd_closed, label='Eyes Closed', alpha=0.8)
axes[1, 0].semilogy(freqs, psd_open, label='Eyes Open', alpha=0.8)
axes[1, 0].set_xlim(0, 40)
axes[1, 0].set_xlabel('Frequency (Hz)')
axes[1, 0].set_ylabel('PSD (μV²/Hz)')
axes[1, 0].set_title('Power Spectral Density - P3 Channel')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 3. Topographic map of alpha power
alpha_band = (8, 12)
alpha_indices = (freqs >= alpha_band[0]) & (freqs <= alpha_band[1])

alpha_power_closed = np.zeros(n_channels)
alpha_power_open = np.zeros(n_channels)

for ch in range(n_channels):
    # Eyes closed
    freqs_ch, psd_ch = welch(eeg_data[ch, :, :30].mean(axis=1), sampling_rate, nperseg=sampling_rate*2)
    alpha_power_closed[ch] = np.trapz(psd_ch[alpha_indices], freqs_ch[alpha_indices])
    
    # Eyes open
    freqs_ch, psd_ch = welch(eeg_data[ch, :, 30:].mean(axis=1), sampling_rate, nperseg=sampling_rate*2)
    alpha_power_open[ch] = np.trapz(psd_ch[alpha_indices], freqs_ch[alpha_indices])

# Bar plot of alpha power by channel
x = np.arange(n_channels)
width = 0.35

axes[1, 1].bar(x - width/2, alpha_power_closed, width, label='Eyes Closed', alpha=0.8)
axes[1, 1].bar(x + width/2, alpha_power_open, width, label='Eyes Open', alpha=0.8)
axes[1, 1].set_xlabel('Channel')
axes[1, 1].set_ylabel('Alpha Power (μV²)')
axes[1, 1].set_title('Alpha Power by Channel (8-12 Hz)')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(channel_names, rotation=45)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# === Statistical Analysis ===
print("\n=== Statistical Analysis ===")

# Compare alpha power between conditions
print("Alpha power comparison:")
print(f"Eyes Closed - Mean: {alpha_power_closed.mean():.3f}, Std: {alpha_power_closed.std():.3f}")
print(f"Eyes Open - Mean: {alpha_power_open.mean():.3f}, Std: {alpha_power_open.std():.3f}")

# Effect size (Cohen's d)
pooled_std = np.sqrt(((alpha_power_closed.std()**2) + (alpha_power_open.std()**2)) / 2)
cohens_d = (alpha_power_closed.mean() - alpha_power_open.mean()) / pooled_std
print(f"Effect size (Cohen's d): {cohens_d:.3f}")

# Find channels with largest differences
alpha_diff = alpha_power_closed - alpha_power_open
max_diff_channel = np.argmax(alpha_diff)
print(f"\nLargest alpha difference in channel: {channel_names[max_diff_channel]} ({alpha_diff[max_diff_channel]:.3f} μV²)")

# Correlation between channels
print("\n=== Channel Connectivity Analysis ===")

# Compute average correlation matrices for each condition
corr_closed = np.zeros((n_channels, n_channels))
corr_open = np.zeros((n_channels, n_channels))

for trial in range(30):
    corr_closed += np.corrcoef(eeg_data[:, :, trial])
    corr_open += np.corrcoef(eeg_data[:, :, trial + 30])

corr_closed /= 30
corr_open /= 30

# Plot correlation matrices
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

im1 = axes[0].imshow(corr_closed, cmap='coolwarm', vmin=-1, vmax=1)
axes[0].set_title('Eyes Closed - Correlation Matrix')
axes[0].set_xticks(range(n_channels))
axes[0].set_yticks(range(n_channels))
axes[0].set_xticklabels(channel_names, rotation=45)
axes[0].set_yticklabels(channel_names)
plt.colorbar(im1, ax=axes[0])

im2 = axes[1].imshow(corr_open, cmap='coolwarm', vmin=-1, vmax=1)
axes[1].set_title('Eyes Open - Correlation Matrix')
axes[1].set_xticks(range(n_channels))
axes[1].set_yticks(range(n_channels))
axes[1].set_xticklabels(channel_names, rotation=45)
axes[1].set_yticklabels(channel_names)
plt.colorbar(im2, ax=axes[1])

# Difference matrix
corr_diff = corr_closed - corr_open
im3 = axes[2].imshow(corr_diff, cmap='RdBu_r', vmin=-0.5, vmax=0.5)
axes[2].set_title('Correlation Difference (Closed - Open)')
axes[2].set_xticks(range(n_channels))
axes[2].set_yticks(range(n_channels))
axes[2].set_xticklabels(channel_names, rotation=45)
axes[2].set_yticklabels(channel_names)
plt.colorbar(im3, ax=axes[2])

plt.tight_layout()
plt.show()

# Summary statistics
avg_corr_closed = np.mean(corr_closed[np.triu_indices(n_channels, k=1)])
avg_corr_open = np.mean(corr_open[np.triu_indices(n_channels, k=1)])

print(f"Average correlation - Eyes Closed: {avg_corr_closed:.3f}")
print(f"Average correlation - Eyes Open: {avg_corr_open:.3f}")
print(f"Difference: {avg_corr_closed - avg_corr_open:.3f}")

# 🧠 Knowledge Checkpoints & Interactive Assessments

print("🎯 Learning Checkpoint: Test Your Understanding!")
print("=" * 50)

import numpy as np
from IPython.display import display, HTML

def create_checkpoint(question, options, correct_answer, explanation):
    """Create an interactive checkpoint question"""
    
    print(f"\n❓ {question}")
    print("-" * len(question))
    
    for i, option in enumerate(options, 1):
        print(f"{i}. {option}")
    
    # Simple text-based interaction
    print(f"\n💡 Think about your answer, then run the next cell to see the solution!")
    
    return correct_answer, explanation

def reveal_answer(correct_answer, explanation):
    """Reveal the answer with explanation"""
    print(f"✅ Correct Answer: {correct_answer}")
    print(f"📚 Explanation: {explanation}")

# Checkpoint 1: Array Indexing
q1_correct, q1_explanation = create_checkpoint(
    "You have EEG data with shape (64, 1000, 100) representing (channels, time, trials). You want to select channels [0,1,2], all time points, and trials [10,20,30]. Which approach is correct?",
    [
        "eeg_data[[0,1,2], :, [10,20,30]]",
        "eeg_data[np.ix_([0,1,2], range(1000), [10,20,30])]",
        "eeg_data[0:3, :, 10:31:10]",
        "eeg_data[(0,1,2), :, (10,20,30)]"
    ],
    "2. eeg_data[np.ix_([0,1,2], range(1000), [10,20,30])]",
    "Using np.ix_ creates the proper mesh grid for advanced indexing with different list lengths on different axes. Option 1 would cause a broadcasting error."
)

print("\n" + "="*60)

# Checkpoint 2: Performance 
q2_correct, q2_explanation = create_checkpoint(
    "Which operation will be fastest for computing the mean of a 1000×1000 array?",
    [
        "Using nested for loops in Python",
        "Using np.mean() with the entire array", 
        "Using list comprehension with np.mean on each row",
        "Using pandas DataFrame.mean()"
    ],
    "2. Using np.mean() with the entire array",
    "NumPy's vectorized operations are optimized with compiled C code and SIMD instructions, making them 10-100x faster than pure Python loops."
)

print("\n" + "="*60)

# Checkpoint 3: Neuroscience Application
q3_correct, q3_explanation = create_checkpoint(
    "You're analyzing alpha waves (8-13 Hz) in EEG data sampled at 250 Hz. After computing the FFT, which frequency indices correspond to the alpha band?",
    [
        "indices 8 to 13",
        "indices 16 to 26 (approximately)", 
        "indices 2 to 3.25",
        "indices 32 to 52"
    ],
    "2. indices 16 to 26 (approximately)",
    "For FFT with N samples at sampling rate fs, frequency resolution is fs/N. For alpha band 8-13 Hz at 250 Hz sampling: 8/(250/N) to 13/(250/N). With typical N=1024, this gives approximately indices 33-54, but with shorter windows, indices 16-26 is reasonable."
)

print("\n" + "="*60)

print("🏆 PRACTICAL CODING CHALLENGES")
print("=" * 35)

def challenge_1():
    """Challenge: Create and manipulate realistic EEG data"""
    
    print("\n🥇 CHALLENGE 1: EEG Data Simulation")
    print("-" * 35)
    print("Task: Create realistic EEG data and extract features")
    print("\nRequirements:")
    print("1. Generate 32 channels × 2500 timepoints × 20 trials")
    print("2. Add alpha waves (10 Hz) with amplitude 50 μV")
    print("3. Add noise with standard deviation 25 μV") 
    print("4. Extract alpha power for each channel")
    print("5. Find the channel with highest alpha power")
    
    print("\n💻 Try to solve this yourself, then check the solution below!")
    
    # Solution (students should try first)
    print("\n🔍 SOLUTION:")
    
    # 1. Generate data structure
    n_channels, n_timepoints, n_trials = 32, 2500, 20
    sampling_rate = 250  # Hz
    duration = n_timepoints / sampling_rate  # 10 seconds
    
    eeg_data = np.zeros((n_channels, n_timepoints, n_trials))
    time_axis = np.linspace(0, duration, n_timepoints)
    
    # 2. Add alpha waves and noise
    for trial in range(n_trials):
        for channel in range(n_channels):
            # Alpha wave with slight frequency variation
            alpha_freq = 10 + np.random.normal(0, 0.5)
            alpha_signal = 50 * np.sin(2 * np.pi * alpha_freq * time_axis)
            
            # Add noise
            noise = 25 * np.random.randn(n_timepoints)
            
            eeg_data[channel, :, trial] = alpha_signal + noise
    
    # 3. Extract alpha power using Welch's method
    from scipy.signal import welch
    
    alpha_powers = []
    for channel in range(n_channels):
        channel_powers = []
        for trial in range(n_trials):
            freqs, psd = welch(eeg_data[channel, :, trial], fs=sampling_rate, nperseg=512)
            alpha_mask = (freqs >= 8) & (freqs <= 13)
            alpha_power = np.trapz(psd[alpha_mask], freqs[alpha_mask])
            channel_powers.append(alpha_power)
        
        alpha_powers.append(np.mean(channel_powers))
    
    alpha_powers = np.array(alpha_powers)
    
    # 4. Find channel with highest alpha power
    max_alpha_channel = np.argmax(alpha_powers)
    
    print(f"✅ Generated EEG data shape: {eeg_data.shape}")
    print(f"✅ Alpha powers computed for {n_channels} channels")
    print(f"✅ Channel {max_alpha_channel} has highest alpha power: {alpha_powers[max_alpha_channel]:.2f}")
    print(f"✅ Average alpha power across channels: {np.mean(alpha_powers):.2f} ± {np.std(alpha_powers):.2f}")
    
    return eeg_data, alpha_powers

def challenge_2():
    """Challenge: Error debugging"""
    
    print("\n🥈 CHALLENGE 2: Debug the Buggy Code")
    print("-" * 40)
    print("The following code has several bugs. Can you spot and fix them?")
    
    buggy_code = '''
# Buggy EEG analysis code
def analyze_eeg_buggy(eeg_data):
    # Bug 1: Wrong axis for time averaging
    time_avg = np.mean(eeg_data, axis=0)  # Should be axis=1
    
    # Bug 2: Division by zero potential
    normalized = eeg_data / np.std(eeg_data, axis=1)  # Missing keepdims=True
    
    # Bug 3: Wrong indexing
    channels = [0, 1, 2]
    trials = [5, 10, 15, 20]
    subset = eeg_data[channels, :, trials]  # Broadcasting error
    
    # Bug 4: Inefficient memory usage
    result1 = eeg_data ** 2
    result2 = np.sqrt(result1)
    result3 = result1 + result2
    final = np.mean(result3)
    
    return final
'''
    
    print("🐛 Buggy Code:")
    print(buggy_code)
    
    print("\n🔧 CORRECTED VERSION:")
    
    def analyze_eeg_fixed(eeg_data):
        """Fixed version of the EEG analysis"""
        
        # Fix 1: Correct axis for time averaging (channels × time × trials)
        time_avg = np.mean(eeg_data, axis=1)  # Average over time (axis=1)
        
        # Fix 2: Add keepdims to prevent broadcasting issues
        normalized = eeg_data / np.std(eeg_data, axis=1, keepdims=True)
        
        # Fix 3: Use np.ix_ for proper advanced indexing
        channels = [0, 1, 2]
        trials = [5, 10, 15, 20]
        subset = eeg_data[np.ix_(channels, range(eeg_data.shape[1]), trials)]
        
        # Fix 4: Chain operations to save memory
        final = np.mean(eeg_data**2 + np.sqrt(eeg_data**2))
        
        return final, time_avg, normalized, subset
    
    # Test with sample data
    test_data = np.random.randn(32, 1000, 50) * 50
    
    try:
        result, time_avg, normalized, subset = analyze_eeg_fixed(test_data)
        print("✅ Fixed code runs successfully!")
        print(f"   Final result: {result:.3f}")
        print(f"   Time average shape: {time_avg.shape}")
        print(f"   Subset shape: {subset.shape}")
        print(f"   Normalized data std: {np.std(normalized):.3f}")
    except Exception as e:
        print(f"❌ Error in fixed code: {e}")

def challenge_3():
    """Challenge: Performance optimization"""
    
    print("\n🥉 CHALLENGE 3: Optimize This Function")
    print("-" * 40)
    print("Make this slow function faster:")
    
    def slow_correlation_analysis(eeg_data):
        """Slow version - compute correlation between all channel pairs"""
        n_channels = eeg_data.shape[0]
        correlations = np.zeros((n_channels, n_channels))
        
        for i in range(n_channels):
            for j in range(n_channels):
                # Compute correlation coefficient manually (slow!)
                mean_i = np.mean(eeg_data[i])
                mean_j = np.mean(eeg_data[j])
                
                numerator = np.sum((eeg_data[i] - mean_i) * (eeg_data[j] - mean_j))
                denom_i = np.sqrt(np.sum((eeg_data[i] - mean_i)**2))
                denom_j = np.sqrt(np.sum((eeg_data[j] - mean_j)**2))
                
                correlations[i, j] = numerator / (denom_i * denom_j)
        
        return correlations
    
    def fast_correlation_analysis(eeg_data):
        """Optimized version using NumPy's built-in function"""
        # One line vs nested loops!
        return np.corrcoef(eeg_data)
    
    # Performance comparison
    test_data = np.random.randn(16, 2500) * 50  # Smaller for demo
    
    print("🐌 Timing slow version...")
    start_time = time.time()
    slow_result = slow_correlation_analysis(test_data)
    slow_time = time.time() - start_time
    
    print("⚡ Timing fast version...")
    start_time = time.time()
    fast_result = fast_correlation_analysis(test_data)
    fast_time = time.time() - start_time
    
    print(f"\n📊 Performance Results:")
    print(f"   Slow version: {slow_time:.4f} seconds")
    print(f"   Fast version: {fast_time:.4f} seconds")
    print(f"   Speedup: {slow_time/fast_time:.1f}x faster!")
    print(f"   Results match: {np.allclose(slow_result, fast_result)}")

print("📝 SELF-ASSESSMENT CHECKLIST")
print("=" * 30)

checklist_items = [
    "✅ I can create and manipulate multi-dimensional NumPy arrays",
    "✅ I understand array indexing and slicing for neuroscience data",
    "✅ I can use vectorized operations instead of loops",
    "✅ I know how to debug common array shape and indexing errors",
    "✅ I can compute basic statistics along different axes",
    "✅ I understand the importance of performance optimization",
    "✅ I can validate and troubleshoot EEG data",
    "✅ I'm comfortable with linear algebra operations in NumPy"
]

print("\nCheck off the items you feel confident about:")
for item in checklist_items:
    print(f"   {item}")

print("\n💡 If you checked less than 6 items, review the corresponding sections!")

# Run the challenges
challenge_1()
challenge_2() 
challenge_3()

print("\n\n🎓 CONGRATULATIONS!")
print("You've completed the Python & NumPy fundamentals for neuroscience!")
print("🚀 You're ready to move on to machine learning concepts!")

## Exercise 1: Event-Related Potential (ERP) Analysis 🎯

**Task**: Analyze event-related potentials by extracting epochs around stimulus events and computing averaged responses.

**Background**: In EEG experiments, we often want to see how the brain responds to specific stimuli (like seeing a face or hearing a tone). We extract small time windows around each stimulus and average across trials to reveal the typical brain response.

In [ ]:
# === Exercise 1: Event-Related Potential Analysis ===
print("=== Exercise 1: ERP Analysis ===")

# Simulate continuous EEG data
sampling_rate = 250  # Hz
duration = 120  # seconds (2 minutes)
n_channels = 4
channel_names = ['Fz', 'Cz', 'Pz', 'Oz']

# Generate continuous EEG
np.random.seed(123)
time_continuous = np.linspace(0, duration, int(sampling_rate * duration))
continuous_eeg = np.random.randn(n_channels, len(time_continuous)) * 2

# Add some background alpha rhythm
for ch in range(n_channels):
    continuous_eeg[ch] += 3 * np.sin(2 * np.pi * 10 * time_continuous + ch * 0.5)

# Generate stimulus events (random times)
n_events = 50
event_times = np.sort(np.random.uniform(5, duration-5, n_events))  # Avoid edges
event_samples = (event_times * sampling_rate).astype(int)

# Add stimulus-locked responses to the data
for event_sample in event_samples:
    # Create a stereotypical ERP response
    erp_time = np.linspace(0, 1, 250)  # 1 second response
    
    # N100 component (negative deflection at 100ms)
    n100 = -5 * np.exp(-((erp_time - 0.1)**2) / (2 * 0.02**2))
    
    # P300 component (positive deflection at 300ms)
    p300 = 8 * np.exp(-((erp_time - 0.3)**2) / (2 * 0.05**2))
    
    erp_response = n100 + p300
    
    # Add to each channel (with some variability)
    for ch in range(n_channels):
        if event_sample + 250 < continuous_eeg.shape[1]:  # Make sure we don't go out of bounds
            # Different channels have different sensitivities
            sensitivity = [0.5, 1.0, 1.2, 0.8][ch]  # Cz and Pz are most sensitive
            continuous_eeg[ch, event_sample:event_sample+250] += sensitivity * erp_response

print(f"Continuous EEG shape: {continuous_eeg.shape}")
print(f"Number of events: {n_events}")
print(f"Event times (first 10): {event_times[:10]}")

# YOUR TASK: Complete the following functions

def extract_epochs(continuous_data, event_samples, pre_stim=0.2, post_stim=0.8, sampling_rate=250):
    """
    Extract epochs around stimulus events.
    
    Parameters:
    -----------
    continuous_data : array, shape (n_channels, n_timepoints)
        Continuous EEG data
    event_samples : array
        Sample indices of stimulus events
    pre_stim : float
        Time before stimulus (seconds)
    post_stim : float
        Time after stimulus (seconds)
    sampling_rate : int
        Sampling frequency
        
    Returns:
    --------
    epochs : array, shape (n_channels, n_timepoints_epoch, n_events)
        Extracted epochs
    epoch_times : array
        Time axis for epochs (relative to stimulus)
    """
    # TODO: Implement epoch extraction
    pre_samples = int(pre_stim * sampling_rate)
    post_samples = int(post_stim * sampling_rate)
    epoch_length = pre_samples + post_samples
    
    n_channels = continuous_data.shape[0]
    valid_events = []
    epochs_list = []
    
    for event_sample in event_samples:
        start_sample = event_sample - pre_samples
        end_sample = event_sample + post_samples
        
        # Check if epoch is within data bounds
        if start_sample >= 0 and end_sample < continuous_data.shape[1]:
            epoch = continuous_data[:, start_sample:end_sample]
            epochs_list.append(epoch)
            valid_events.append(event_sample)
    
    epochs = np.stack(epochs_list, axis=2)
    epoch_times = np.linspace(-pre_stim, post_stim, epoch_length)
    
    return epochs, epoch_times

def baseline_correct_epochs(epochs, baseline_period, epoch_times):
    """
    Apply baseline correction to epochs.
    
    Parameters:
    -----------
    epochs : array, shape (n_channels, n_timepoints, n_events)
        Epoch data
    baseline_period : tuple
        (start_time, end_time) for baseline period
    epoch_times : array
        Time axis for epochs
        
    Returns:
    --------
    epochs_corrected : array
        Baseline-corrected epochs
    """
    # TODO: Implement baseline correction
    baseline_mask = (epoch_times >= baseline_period[0]) & (epoch_times <= baseline_period[1])
    baseline_mean = np.mean(epochs[:, baseline_mask, :], axis=1, keepdims=True)
    epochs_corrected = epochs - baseline_mean
    
    return epochs_corrected

def compute_erp(epochs):
    """
    Compute event-related potential by averaging across trials.
    
    Parameters:
    -----------
    epochs : array, shape (n_channels, n_timepoints, n_events)
        Epoch data
        
    Returns:
    --------
    erp : array, shape (n_channels, n_timepoints)
        Averaged ERP
    erp_std : array, shape (n_channels, n_timepoints)
        Standard deviation across trials
    """
    # TODO: Implement ERP computation
    erp = np.mean(epochs, axis=2)
    erp_std = np.std(epochs, axis=2)
    
    return erp, erp_std

# Test your implementation
epochs, epoch_times = extract_epochs(continuous_eeg, event_samples, pre_stim=0.2, post_stim=0.8)
epochs_corrected = baseline_correct_epochs(epochs, baseline_period=(-0.2, 0), epoch_times=epoch_times)
erp, erp_std = compute_erp(epochs_corrected)

print(f"\nResults:")
print(f"Epochs shape: {epochs.shape}")
print(f"ERP shape: {erp.shape}")
print(f"Epoch time range: {epoch_times[0]:.3f} to {epoch_times[-1]:.3f} seconds")

# Plot results
plt.figure(figsize=(12, 8))

for ch in range(n_channels):
    plt.subplot(2, 2, ch + 1)
    plt.plot(epoch_times, erp[ch], 'b-', linewidth=2, label='ERP')
    plt.fill_between(epoch_times, erp[ch] - erp_std[ch], erp[ch] + erp_std[ch], 
                     alpha=0.3, color='blue', label='±1 SD')
    plt.axvline(0, color='red', linestyle='--', alpha=0.7, label='Stimulus')
    plt.axhline(0, color='black', linestyle='-', alpha=0.3)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude (μV)')
    plt.title(f'ERP - Channel {channel_names[ch]}')
    plt.grid(True, alpha=0.3)
    if ch == 0:
        plt.legend()

plt.tight_layout()
plt.show()

# Analysis questions for you to think about:
print("\n=== Analysis Questions ===")
print("1. Which channel shows the strongest P300 response?")
print("2. At what time does the N100 component peak?")
print("3. How does the signal-to-noise ratio compare across channels?")

# Find P300 peak
p300_window = (epoch_times >= 0.2) & (epoch_times <= 0.4)
p300_peaks = np.argmax(erp[:, p300_window], axis=1)
p300_peak_times = epoch_times[p300_window][p300_peaks]

print(f"\nP300 peak times by channel:")
for ch in range(n_channels):
    print(f"  {channel_names[ch]}: {p300_peak_times[ch]:.3f} s")

## Exercise 2: Spectral Analysis and Connectivity 🌊

**Task**: Analyze the frequency content of neural signals and compute connectivity between brain regions.

**Background**: The brain operates at different frequency bands (delta, theta, alpha, beta, gamma). Understanding how these frequencies change and how they're synchronized between brain regions gives us insights into brain function.

In [ ]:
# === Exercise 2: Spectral Analysis and Connectivity ===
print("=== Exercise 2: Spectral Analysis and Connectivity ===")

# Generate sample data with known connectivity
np.random.seed(456)
sampling_rate = 500  # Hz
duration = 60  # seconds
n_channels = 6
channel_names = ['F3', 'F4', 'C3', 'C4', 'P3', 'P4']

time_axis = np.linspace(0, duration, int(sampling_rate * duration))
n_timepoints = len(time_axis)

# Create signals with known connectivity patterns
signals = np.zeros((n_channels, n_timepoints))

# Generate base oscillations
alpha_source = np.sin(2 * np.pi * 10 * time_axis)  # 10 Hz alpha
beta_source = np.sin(2 * np.pi * 20 * time_axis)   # 20 Hz beta
gamma_source = np.sin(2 * np.pi * 40 * time_axis)  # 40 Hz gamma

# Add noise and connectivity patterns
for ch in range(n_channels):
    # Base noise
    signals[ch] = 0.5 * np.random.randn(n_timepoints)
    
    # Add frequency-specific connectivity
    if 'F' in channel_names[ch]:  # Frontal channels
        signals[ch] += 2 * beta_source + 0.5 * np.random.randn(n_timepoints)
    elif 'C' in channel_names[ch]:  # Central channels
        signals[ch] += 1.5 * alpha_source + 1 * beta_source + 0.5 * np.random.randn(n_timepoints)
    elif 'P' in channel_names[ch]:  # Posterior channels
        signals[ch] += 3 * alpha_source + 0.5 * np.random.randn(n_timepoints)
    
    # Add some gamma coupling
    if ch % 2 == 0:  # Left hemisphere
        signals[ch] += 0.8 * gamma_source
    else:  # Right hemisphere
        # Delayed gamma (simulate inter-hemispheric delay)
        delay_samples = int(0.01 * sampling_rate)  # 10ms delay
        delayed_gamma = np.zeros_like(gamma_source)
        delayed_gamma[delay_samples:] = gamma_source[:-delay_samples]
        signals[ch] += 0.8 * delayed_gamma

print(f"Generated signals shape: {signals.shape}")
print(f"Duration: {duration} seconds")
print(f"Sampling rate: {sampling_rate} Hz")

# YOUR TASK: Implement the following functions

def compute_power_spectrum(data, sampling_rate, nperseg=None):
    """
    Compute power spectral density for multi-channel data.
    
    Parameters:
    -----------
    data : array, shape (n_channels, n_timepoints)
        Multi-channel signal data
    sampling_rate : int
        Sampling frequency
    nperseg : int, optional
        Length of each segment for Welch's method
        
    Returns:
    --------
    frequencies : array
        Frequency values
    psd : array, shape (n_channels, n_frequencies)
        Power spectral density for each channel
    """
    # TODO: Implement power spectrum computation
    if nperseg is None:
        nperseg = sampling_rate * 4  # 4 second windows
    
    n_channels = data.shape[0]
    
    # Compute PSD for first channel to get frequency array
    frequencies, psd_ch = signal.welch(data[0], sampling_rate, nperseg=nperseg)
    
    # Initialize PSD array
    psd = np.zeros((n_channels, len(frequencies)))
    
    # Compute PSD for each channel
    for ch in range(n_channels):
        frequencies, psd[ch] = signal.welch(data[ch], sampling_rate, nperseg=nperseg)
    
    return frequencies, psd

def compute_band_power(psd, frequencies, freq_bands):
    """
    Compute power in specific frequency bands.
    
    Parameters:
    -----------
    psd : array, shape (n_channels, n_frequencies)
        Power spectral density
    frequencies : array
        Frequency values
    freq_bands : dict
        Dictionary of frequency bands {band_name: (low_freq, high_freq)}
        
    Returns:
    --------
    band_powers : dict
        Dictionary of band powers {band_name: array of powers per channel}
    """
    # TODO: Implement band power computation
    band_powers = {}
    
    for band_name, (low_freq, high_freq) in freq_bands.items():
        # Create frequency mask
        freq_mask = (frequencies >= low_freq) & (frequencies <= high_freq)
        
        # Compute power in band for each channel
        band_power = np.zeros(psd.shape[0])
        for ch in range(psd.shape[0]):
            band_power[ch] = np.trapz(psd[ch, freq_mask], frequencies[freq_mask])
        
        band_powers[band_name] = band_power
    
    return band_powers

def compute_coherence(data, sampling_rate, nperseg=None):
    """
    Compute coherence between all pairs of channels.
    
    Parameters:
    -----------
    data : array, shape (n_channels, n_timepoints)
        Multi-channel signal data
    sampling_rate : int
        Sampling frequency
    nperseg : int, optional
        Length of each segment
        
    Returns:
    --------
    frequencies : array
        Frequency values
    coherence : array, shape (n_channels, n_channels, n_frequencies)
        Coherence matrix
    """
    # TODO: Implement coherence computation
    if nperseg is None:
        nperseg = sampling_rate * 4
    
    n_channels = data.shape[0]
    
    # Compute coherence for first pair to get frequency array
    frequencies, coh_temp = signal.coherence(data[0], data[1], sampling_rate, nperseg=nperseg)
    
    # Initialize coherence array
    coherence = np.zeros((n_channels, n_channels, len(frequencies)))
    
    # Compute coherence for all pairs
    for i in range(n_channels):
        for j in range(n_channels):
            if i == j:
                coherence[i, j, :] = 1.0  # Perfect coherence with self
            else:
                frequencies, coherence[i, j, :] = signal.coherence(data[i], data[j], sampling_rate, nperseg=nperseg)
    
    return frequencies, coherence

# Test your implementation
frequencies, psd = compute_power_spectrum(signals, sampling_rate)

freq_bands = {
    'delta': (1, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 80)
}

band_powers = compute_band_power(psd, frequencies, freq_bands)
coh_frequencies, coherence = compute_coherence(signals, sampling_rate)

print(f"\nResults:")
print(f"PSD shape: {psd.shape}")
print(f"Frequency range: {frequencies[0]:.1f} to {frequencies[-1]:.1f} Hz")
print(f"Coherence shape: {coherence.shape}")

# Plot results
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Power spectra
for ch in range(n_channels):
    axes[0, 0].semilogy(frequencies, psd[ch], label=channel_names[ch], alpha=0.8)
axes[0, 0].set_xlim(0, 60)
axes[0, 0].set_xlabel('Frequency (Hz)')
axes[0, 0].set_ylabel('PSD (μV²/Hz)')
axes[0, 0].set_title('Power Spectral Density')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Band powers
band_names = list(freq_bands.keys())
x_pos = np.arange(len(band_names))
bar_width = 0.1

for ch in range(n_channels):
    powers = [band_powers[band][ch] for band in band_names]
    axes[0, 1].bar(x_pos + ch * bar_width, powers, bar_width, 
                   label=channel_names[ch], alpha=0.8)

axes[0, 1].set_xlabel('Frequency Band')
axes[0, 1].set_ylabel('Power (μV²)')
axes[0, 1].set_title('Band Powers')
axes[0, 1].set_xticks(x_pos + bar_width * 2.5)
axes[0, 1].set_xticklabels(band_names)
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Coherence heatmap (average across frequencies)
avg_coherence = np.mean(coherence, axis=2)
im = axes[0, 2].imshow(avg_coherence, cmap='viridis', vmin=0, vmax=1)
axes[0, 2].set_title('Average Coherence Matrix')
axes[0, 2].set_xticks(range(n_channels))
axes[0, 2].set_yticks(range(n_channels))
axes[0, 2].set_xticklabels(channel_names)
axes[0, 2].set_yticklabels(channel_names)
plt.colorbar(im, ax=axes[0, 2])

# 4. Alpha coherence network
alpha_idx = (coh_frequencies >= 8) & (coh_frequencies <= 13)
alpha_coherence = np.mean(coherence[:, :, alpha_idx], axis=2)
im2 = axes[1, 0].imshow(alpha_coherence, cmap='viridis', vmin=0, vmax=1)
axes[1, 0].set_title('Alpha Band Coherence (8-13 Hz)')
axes[1, 0].set_xticks(range(n_channels))
axes[1, 0].set_yticks(range(n_channels))
axes[1, 0].set_xticklabels(channel_names)
axes[1, 0].set_yticklabels(channel_names)
plt.colorbar(im2, ax=axes[1, 0])

# 5. Beta coherence network
beta_idx = (coh_frequencies >= 13) & (coh_frequencies <= 30)
beta_coherence = np.mean(coherence[:, :, beta_idx], axis=2)
im3 = axes[1, 1].imshow(beta_coherence, cmap='viridis', vmin=0, vmax=1)
axes[1, 1].set_title('Beta Band Coherence (13-30 Hz)')
axes[1, 1].set_xticks(range(n_channels))
axes[1, 1].set_yticks(range(n_channels))
axes[1, 1].set_xticklabels(channel_names)
axes[1, 1].set_yticklabels(channel_names)
plt.colorbar(im3, ax=axes[1, 1])

# 6. Coherence spectrum between F3 and F4
f3_idx = channel_names.index('F3')
f4_idx = channel_names.index('F4')
axes[1, 2].plot(coh_frequencies, coherence[f3_idx, f4_idx, :], 'b-', linewidth=2)
axes[1, 2].set_xlim(0, 60)
axes[1, 2].set_ylim(0, 1)
axes[1, 2].set_xlabel('Frequency (Hz)')
axes[1, 2].set_ylabel('Coherence')
axes[1, 2].set_title('F3-F4 Coherence Spectrum')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Analysis summary
print("\n=== Analysis Summary ===")
print("Band power analysis:")
for band in band_names:
    max_channel = np.argmax(band_powers[band])
    print(f"  {band}: strongest in {channel_names[max_channel]} ({band_powers[band][max_channel]:.3f} μV²)")

print("\nCoherence analysis:")
# Find strongest coherence pairs (excluding self-connections)
coherence_no_diag = avg_coherence.copy()
np.fill_diagonal(coherence_no_diag, 0)
max_coherence_idx = np.unravel_index(np.argmax(coherence_no_diag), coherence_no_diag.shape)
print(f"  Strongest coherence: {channel_names[max_coherence_idx[0]]}-{channel_names[max_coherence_idx[1]]} ({coherence_no_diag[max_coherence_idx]:.3f})")

# Interhemispheric coherence
left_channels = [i for i, ch in enumerate(channel_names) if '3' in ch]
right_channels = [i for i, ch in enumerate(channel_names) if '4' in ch]
interhemispheric_coherence = []

for l_ch in left_channels:
    for r_ch in right_channels:
        if channel_names[l_ch][0] == channel_names[r_ch][0]:  # Same region
            coh_val = avg_coherence[l_ch, r_ch]
            interhemispheric_coherence.append(coh_val)
            print(f"  {channel_names[l_ch]}-{channel_names[r_ch]} coherence: {coh_val:.3f}")

print(f"  Average interhemispheric coherence: {np.mean(interhemispheric_coherence):.3f}")

# 🎯 Key Takeaways: What You've Learned

Congratulations! You've just completed a comprehensive review of Python and NumPy fundamentals for neuroscience. Here's what you've mastered:

## 🐍 Python Skills
- **List comprehensions** for efficient data processing
- **Functions** for building reusable analysis tools
- **Classes** for organizing complex analysis pipelines
- **Best practices** for scientific computing

## 🔢 NumPy Mastery
- **Array creation and manipulation** for multi-dimensional brain data
- **Indexing and slicing** to extract specific channels, time windows, and trials
- **Vectorized operations** for lightning-fast computations
- **Mathematical operations** essential for signal processing
- **Linear algebra** foundations for neural networks

## 🧠 Neuroscience Applications
- **EEG data processing** with realistic multi-channel signals
- **Event-related potential (ERP) analysis** for studying brain responses
- **Spectral analysis** to understand frequency content
- **Connectivity analysis** using coherence measures
- **Statistical analysis** of neural data

## 💡 Performance Tips You've Learned

1. **Always use vectorized operations** instead of Python loops
2. **Specify axis parameters** carefully when working with multi-dimensional data
3. **Use keepdims=True** when you need to preserve array dimensions
4. **Leverage NumPy's broadcasting** for efficient operations
5. **Choose appropriate data types** to save memory

---

## 🚀 What's Next?

In the next notebook (`02_ml_foundations.ipynb`), you'll build on these fundamentals to explore:

- **Machine learning concepts** relevant to neuroscience
- **Classification and regression** for brain state detection
- **Cross-validation** and model evaluation
- **Feature engineering** for neural signals
- **Preparing for deep learning** with PyTorch

You're well-equipped for this journey! The NumPy skills you've practiced here will be your foundation for everything that comes next.

---

**Ready to continue your deep learning journey? See you in the next notebook! 🧠⚡**